# Comprobante OCR - Examen Final Machine Learning - Unidad 2


## EasyOCR

### Instalacion de EasyOCR


In [3]:
# !pip install easyocr

### Comprobar version de EasyOCR

In [4]:
!pip show easyocr

Name: easyocr
Version: 1.7.2
Summary: End-to-End Multi-Lingual Optical Character Recognition (OCR) Solution
Home-page: https://github.com/jaidedai/easyocr
Author: Rakpong Kittinaradorn
Author-email: r.kittinaradorn@gmail.com
License: Apache License 2.0
Location: /opt/homebrew/Caskroom/miniforge/base/envs/torch2/lib/python3.10/site-packages
Requires: ninja, numpy, opencv-python-headless, Pillow, pyclipper, python-bidi, PyYAML, scikit-image, scipy, Shapely, torch, torchvision
Required-by: 


### Probar con un comprobante con EasyOCR (presicion y tiempo de inferencia)

In [1]:
import easyocr
import time
import os
import re
import numpy as np
from pdf2image import convert_from_path
from PIL import Image
from typing import List

def load_images(file_path: str, dpi: int = 300) -> List:
    """Carga imágenes desde PDF o archivos de imagen"""
    SUPPORTED_FORMATS = {'.pdf', '.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp'}
    
    file_ext = os.path.splitext(file_path)[1].lower()
    
    if file_ext not in SUPPORTED_FORMATS:
        raise ValueError(f"❌ Formato no soportado: {file_ext}\n"
                        f"Formatos válidos: {', '.join(SUPPORTED_FORMATS)}")
    
    if file_ext == '.pdf':
        print(f"📄 Convirtiendo PDF a imágenes (DPI: {dpi})...")
        return convert_from_path(file_path, dpi=dpi)
    else:
        print(f"🖼️  Cargando imagen: {os.path.basename(file_path)}")
        imagen = Image.open(file_path)
        return [imagen]

# Configuración
file_path = 'comprobante/WhatsApp Image 2025-10-16 at 21.56.09.jpeg'  # JPEG
# file_path = 'comprobante/boleta.jpg'  # JPEG
# file_path = 'comprobante/factura.png'  # PNG

# Inicializar EasyOCR con español (una sola vez)
print("🔧 Inicializando EasyOCR...")
reader = easyocr.Reader(['es'])

# Cargar imágenes (desde PDF o archivo de imagen)
imagenes = load_images(file_path)

# Procesar cada página/imagen
total_time = 0
all_texts = []

for i, imagen in enumerate(imagenes):
    # Título de página/imagen
    if len(imagenes) > 1:
        print(f'\n--- Página {i+1}/{len(imagenes)} ---')
    else:
        print(f'\n--- Procesando imagen ---')
    
    # Convertir PIL Image a numpy array
    imagen_np = np.array(imagen)
    
    # Realizar OCR y medir tiempo
    start_time = time.time()
    resultados = reader.readtext(imagen_np)
    inference_time = time.time() - start_time
    total_time += inference_time
    
    # Extraer textos detectados
    page_texts = []
    for bbox, texto, confianza in resultados:
        print(f'Texto: {texto}, Confianza: {confianza:.4f}')
        page_texts.append(texto)
    
    all_texts.extend(page_texts)
    print(f'⏱️  Tiempo de inferencia: {inference_time:.2f} segundos')

# Resumen final
print(f'\n{"="*50}')
print(f'=== RESUMEN ===')
print(f'{"="*50}')
print(f'Total de páginas/imágenes: {len(imagenes)}')
print(f'Total de textos detectados: {len(all_texts)}')
print(f'⏱️  Tiempo total: {total_time:.2f} segundos')
print(f'{"="*50}')

🔧 Inicializando EasyOCR...
🖼️  Cargando imagen: WhatsApp Image 2025-10-16 at 21.56.09.jpeg


FileNotFoundError: [Errno 2] No such file or directory: '/Users/franspaxi/machine-learning/unidad_2/examen_final/comprobante/WhatsApp Image 2025-10-16 at 21.56.09.jpeg'

## PaddleOCR

### Instalacion de PaddleOCR

In [ ]:
# !pip install paddleocr

### Probar con un comprobante usando PaddleOCR

In [ ]:
from paddleocr import PaddleOCR
ocr = PaddleOCR(
    use_doc_orientation_classify=False,
    use_doc_unwarping=False,
    use_textline_orientation=False,
    text_det_limit_side_len=1000,
    text_det_limit_type="max",
    lang='es',
)
        
# Run OCR inference on a sample image 
result = ocr.predict(
    input="comprobante/professionaltaxinvoicetemplate.pdf"
    )

# Extraer y mostrar las palabras detectadas
print(f'\n{"="*50}')
print("=== PALABRAS DETECTADAS ===")
print(f'{"="*50}')

for i, res in enumerate(result, 1):
    textos = res['rec_texts']
    confianza = res['rec_scores']
    page = res['page_index']
    for j, texto in enumerate(textos, 1):
        print(f'{j}. text: {texto} - confianza: {confianza[j-1]} - página: {page}')

print(f'\n{"="*50}')

/opt/homebrew/Caskroom/miniforge/base/envs/torch2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/Caskroom/miniforge/base/envs/torch2/lib/python3.10/site-packages/paddle/utils/cpp_extension/extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/franspaxi/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('latin_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory m


=== PALABRAS DETECTADAS ===
1. text: Install Invoice Manager - confianza: 0.9461337924003601 - página: 0
2. text: Date Picker - confianza: 0.9987428188323975 - página: 0
3. text: Fomula Manager - confianza: 0.999896228313446 - página: 0
4. text: Password Remover - confianza: 0.9786750078201294 - página: 0
5. text: from Microsoft Store - confianza: 0.975257396697998 - página: 0
6. text: adds a calendar - confianza: 0.9991549849510193 - página: 0
7. text: Find, update, analyze, import and export - confianza: 0.975149929523468 - página: 0
8. text: to generate PDF invoice - confianza: 0.9980165958404541 - página: 0
9. text: to date cells - confianza: 0.992823600769043 - página: 0
10. text: Excel formulas and defined names - confianza: 0.9794796705245972 - página: 0
11. text: for Excel - confianza: 0.9995501041412354 - página: 0
12. text: Mind Map - confianza: 0.9999340772628784 - página: 0
13. text: Making - confianza: 0.9999475479125977 - página: 0
14. text: Create PDF files at any - con

### Probar con un comprobante usando PADDLE OCR con CÓDIGO LIMPIO

In [ ]:
from dataclasses import dataclass
from paddleocr import PaddleOCR
import re
from typing import List, Optional

# ==================== OCR PROCESSOR ====================

class OCRConfig:
    """Configuración para PaddleOCR"""
    
    def __init__(self, lang: str = 'es'):
        self.use_doc_orientation_classify = False
        self.use_doc_unwarping = False
        self.use_textline_orientation = False
        self.text_det_limit_side_len = 1000
        self.text_det_limit_type = "max"
        self.lang = lang


class OCRProcessor:
    """Procesa OCR en imágenes/PDFs"""
    
    def __init__(self, config: OCRConfig):
        self.ocr = PaddleOCR(
            use_doc_orientation_classify=config.use_doc_orientation_classify,
            use_doc_unwarping=config.use_doc_unwarping,
            use_textline_orientation=config.use_textline_orientation,
            text_det_limit_side_len=config.text_det_limit_side_len,
            text_det_limit_type=config.text_det_limit_type,
            lang=config.lang,
        )
    
    def process(self, input_path: str):
        """Ejecuta OCR en el archivo de entrada"""
        return self.ocr.predict(input=input_path)
    
# ==================== RESULT PRINTER ====================

class ResultPrinter:
    """Responsable de imprimir resultados"""
    
    @staticmethod
    def print_header(title: str):
        """Imprime un encabezado formateado"""
        print(f'\n{"="*50}')
        print(f'=== {title} ===')
        print(f'{"="*50}')
    
    @staticmethod
    def print_ocr_texts(ocr_result):
        """Imprime todos los textos detectados por OCR"""
        ResultPrinter.print_header("PALABRAS DETECTADAS")
        
        for res in ocr_result:
            textos = res['rec_texts']
            confianzas = res['rec_scores']
            page = res['page_index']
            
            for j, texto in enumerate(textos, 1):
                print(f'{j}. text: {texto} - confianza: {confianzas[j-1]:.4f} - página: {page}')
        
        print(f'{"="*50}')

# ==================== MAIN ORCHESTRATOR ====================

class ComprobanteProcessor:
    """Orquestador principal para procesar números en comprobantes"""
    
    def __init__(self, input_path: str):
        self.input_path = input_path
        self.ocr_processor = OCRProcessor(OCRConfig())
        self.printer = ResultPrinter()
    
    def process(self):
        """Procesa el comprobante completo"""
        # 1. Ejecutar OCR
        ocr_result = self.ocr_processor.process(self.input_path)
        
        # 2. Mostrar textos detectados
        self.printer.print_ocr_texts(ocr_result)


# ==================== EXECUTION ====================

if __name__ == '__main__':
    input_path = "comprobante/comprobante sin nombre de cliente (NUBEFACT).png"

    processor = ComprobanteProcessor(input_path)
    processor.process()

## Clasificar el IMPORTE TOTAL del comprobante

In [ ]:
from dataclasses import dataclass
from paddleocr import PaddleOCR
import re
from typing import List, Optional
from abc import ABC, abstractmethod
import concurrent.futures


# ==================== DATA CLASSES ====================

@dataclass
class NumeroDetectado:
    """Información de un número decimal detectado"""
    texto_original: str
    valor_numerico: float
    confianza: float
    pagina: int
    posicion: int


# ==================== PATTERNS & EXTRACTORS ====================

class NumberPattern(ABC):
    """Clase base abstracta para patrones de números"""
    
    @abstractmethod
    def match(self, texto: str) -> Optional[float]:
        """Intenta extraer un número del texto"""
        pass


class ComaSeparadorMilesPattern(NumberPattern):
    """Patrón para formato 1,298.45 (peruano/USA) con S/ opcional"""
    
    PATTERN = r'^(?:S/\s*)?[\$\s]*(\d{1,3}(?:,\d{3})*\.\d{2})$'
    
    def match(self, texto: str) -> Optional[float]:
        match = re.match(self.PATTERN, texto)
        if match:
            numero_str = match.group(1).replace(',', '')
            return float(numero_str)
        return None


class PuntoSeparadorMilesPattern(NumberPattern):
    """Patrón para formato europeo 1.298,45 con S/ opcional"""
    
    PATTERN = r'^(?:S/\s*)?[\$\s]*(\d{1,3}(?:\.\d{3})*,\d{2})$'
    
    def match(self, texto: str) -> Optional[float]:
        match = re.match(self.PATTERN, texto)
        if match:
            numero_str = match.group(1).replace('.', '').replace(',', '.')
            return float(numero_str)
        return None


class SimpleDecimalPattern(NumberPattern):
    """Patrón para formato simple 150.00 con S/ opcional"""
    
    PATTERN = r'^(?:S/\s*)?[\$\s]*(\d+\.\d{2})$'
    
    def match(self, texto: str) -> Optional[float]:
        match = re.match(self.PATTERN, texto)
        if match:
            return float(match.group(1))
        return None


class EnteroConSeparadorPattern(NumberPattern):
    """Patrón para números enteros con separador 1,500 con S/ opcional"""
    
    PATTERN = r'^(?:S/\s*)?[\$\s]*(\d{1,3}(?:,\d{3})*)$'
    
    def match(self, texto: str) -> Optional[float]:
        match = re.match(self.PATTERN, texto)
        if match:
            numero_str = match.group(1).replace(',', '')
            return float(numero_str)
        return None


class DecimalExtractor:
    """Extractor de números decimales usando múltiples patrones"""
    
    def __init__(self):
        self.patterns: List[NumberPattern] = [
            ComaSeparadorMilesPattern(),
            PuntoSeparadorMilesPattern(),
            SimpleDecimalPattern(),
            EnteroConSeparadorPattern()
        ]
    
    def extract(self, texto: str) -> Optional[float]:
        """Intenta extraer un número decimal del texto usando todos los patrones"""
        texto = texto.strip()
        
        for pattern in self.patterns:
            resultado = pattern.match(texto)
            if resultado is not None:
                return resultado
        
        return None


# ==================== OCR PROCESSOR ====================

class OCRConfig:
    """Configuración para PaddleOCR"""
    
    def __init__(self, lang: str = 'es'):
        self.use_doc_orientation_classify = False
        self.use_doc_unwarping = False
        self.use_textline_orientation = False
        self.text_det_limit_side_len = 1000
        self.text_det_limit_type = "max"
        self.lang = lang


class OCRProcessor:
    """Procesa OCR en imágenes/PDFs"""
    
    def __init__(self, config: OCRConfig):
        self.ocr = PaddleOCR(
            use_doc_orientation_classify=config.use_doc_orientation_classify,
            use_doc_unwarping=config.use_doc_unwarping,
            use_textline_orientation=config.use_textline_orientation,
            text_det_limit_side_len=config.text_det_limit_side_len,
            text_det_limit_type=config.text_det_limit_type,
            lang=config.lang,
        )
    
    def process(self, input_path: str):
        """Ejecuta OCR en el archivo de entrada"""
        return self.ocr.predict(input=input_path)


# ==================== NUMBER ANALYZER ====================

class NumberAnalyzer:
    """Analiza y extrae números de resultados OCR"""
    
    def __init__(self, extractor: DecimalExtractor):
        self.extractor = extractor
    
    def analyze(self, ocr_result) -> List[NumeroDetectado]:
        """Analiza resultados OCR y extrae todos los números"""
        numeros_detectados = []
        
        for res in ocr_result:
            textos = res['rec_texts']
            confianzas = res['rec_scores']
            page = res['page_index']
            
            for j, texto in enumerate(textos):
                numero = self.extractor.extract(texto)
                if numero is not None:
                    numeros_detectados.append(NumeroDetectado(
                        texto_original=texto,
                        valor_numerico=numero,
                        confianza=confianzas[j],
                        pagina=page,
                        posicion=j + 1
                    ))
        
        return numeros_detectados


# ==================== RESULT PRINTER ====================

class ResultPrinter:
    """Responsable de imprimir resultados"""
    
    @staticmethod
    def print_header(title: str):
        """Imprime un encabezado formateado"""
        print(f'\n{"="*50}')
        print(f'=== {title} ===')
        print(f'{"="*50}')
    
    @staticmethod
    def print_ocr_texts(ocr_result):
        """Imprime todos los textos detectados por OCR"""
        ResultPrinter.print_header("PALABRAS DETECTADAS")
        
        for res in ocr_result:
            textos = res['rec_texts']
            confianzas = res['rec_scores']
            page = res['page_index']
            
            for j, texto in enumerate(textos, 1):
                print(f'{j}. text: {texto} - confianza: {confianzas[j-1]:.4f} - página: {page}')
        
        print(f'{"="*50}')
    
    @staticmethod
    def print_detected_numbers(numeros: List[NumeroDetectado]):
        """Imprime todos los números decimales encontrados"""
        ResultPrinter.print_header("NÚMEROS DECIMALES ENCONTRADOS")
        
        if not numeros:
            print('⚠️  No se encontraron números decimales')
            print(f'{"="*50}')
            return
        
        for idx, num in enumerate(numeros, 1):
            print(f'{idx}. Texto: "{num.texto_original}" → Valor: {num.valor_numerico:.2f} '
                  f'(Confianza: {num.confianza:.4f}, Página: {num.pagina})')
    
    @staticmethod
    def print_highest_number(numero: NumeroDetectado):
        """Imprime el número más alto detectado"""
        ResultPrinter.print_header("NÚMERO MÁS ALTO DETECTADO")
        print(f'💰 Texto Original: "{numero.texto_original}"')
        print(f'💰 Valor Numérico: {numero.valor_numerico:.2f}')
        print(f'💰 Confianza: {numero.confianza:.4f}')
        print(f'💰 Página: {numero.pagina}')
        print(f'💰 Posición: {numero.posicion}')
        print(f'{"="*50}')


# ==================== MAIN ORCHESTRATOR ====================

class ComprobanteNumberProcessor:
    """Orquestador principal para procesar números en comprobantes"""
    
    def __init__(self, input_path: str):
        self.input_path = input_path
        self.ocr_processor = OCRProcessor(OCRConfig())
        self.number_analyzer = NumberAnalyzer(DecimalExtractor())
        self.printer = ResultPrinter()
    
    def process(self) -> str:  # ✅ Cambiado: retorna str en lugar de NumeroDetectado
        """Procesa el comprobante y retorna el valor más alto como string o cadena vacía"""
        # 1. Ejecutar OCR
        ocr_result = self.ocr_processor.process(self.input_path)
        
        # 2. Mostrar textos detectados
        self.printer.print_ocr_texts(ocr_result)
        
        # 3. Analizar y extraer números
        numeros_detectados = self.number_analyzer.analyze(ocr_result)
        
        # 4. Mostrar números encontrados
        self.printer.print_detected_numbers(numeros_detectados)
        
        # 5. Encontrar y retornar el número más alto
        if numeros_detectados:
            numero_mas_alto = max(numeros_detectados, key=lambda x: x.valor_numerico)
            self.printer.print_highest_number(numero_mas_alto)
            return f'{numero_mas_alto.valor_numerico:.2f}'  # ✅ Retorna como string
        else:
            return ''  # ✅ Retorna cadena vacía si no hay números


# ==================== EXECUTION ====================

if __name__ == '__main__':
    input_path = "comprobantes/images/02_jpg.rf.65eab3cc2fece1483d867777e66a204d.jpg"
    
    processor = ComprobanteNumberProcessor(input_path)
    importe_total = processor.process()  # ✅ Ahora es string
    
    # ✅ Mostrar resultado
    if importe_total:
        print(f'\nNúmero más alto extraído: {importe_total}')
    else:
        print('\n⚠️  No se encontraron números decimales')

Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/franspaxi/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('latin_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/franspaxi/.paddlex/official_models/latin_PP-OCRv5_mobile_rec`.



=== PALABRAS DETECTADAS ===
1. text: MUNDPC - confianza: 0.9997 - página: None
2. text: RUC 20600950453 - confianza: 0.9864 - página: None
3. text: FACTURA - confianza: 1.0000 - página: None
4. text: LINEA DE EQUIPOS DE COMPUTO Y SUMINISTROS - confianza: 0.9958 - página: None
5. text: ELECTRÓNICA - confianza: 0.9999 - página: None
6. text: CORPORACION MUNDO PC S.A.C. - confianza: 0.9961 - página: None
7. text: JR. GONZALES PRADA NRO. 521 , PUNO , SAN ROMÁN , JULIACA - confianza: 0.9888 - página: None
8. text: F001 - N° 00002651 - confianza: 0.9976 - página: None
9. text: TEL: 964703003 - confianza: 0.9998 - página: None
10. text: CORREO: Julia_r_18@hotmail.com - confianza: 0.9973 - página: None
11. text: CLIENTE: - confianza: 0.9999 - página: None
12. text: MUNICIPALIDAD DEL CENTRO POBLADO DE QUICHO - confianza: 0.9967 - página: None
13. text: MONEDA: - confianza: 0.9999 - página: None
14. text: Soles - confianza: 0.9992 - página: None
15. text: RUC: - confianza: 0.9998 - página: None

## Clasificar el DNI para conseguir el NOMBRE COMPLETO del cliente

In [6]:
from dataclasses import dataclass
from paddleocr import PaddleOCR
import re
import time
from typing import List, Optional, Dict, Tuple
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


# ==================== DATA CLASSES ====================

@dataclass
class DNIInfo:
    """Información del DNI encontrado"""
    numero: str
    confianza: float
    pagina: int
    posicion: int


@dataclass
class PersonaInfo:
    """Información completa de una persona"""
    dni: DNIInfo
    nombre_completo: Optional[str]


# ==================== OCR CONFIGURATION ====================

class OCRConfig:
    """Configuración para PaddleOCR"""
    
    def __init__(self, lang: str = 'es'):
        self.use_doc_orientation_classify = False
        self.use_doc_unwarping = False
        self.use_textline_orientation = False
        self.text_det_limit_side_len = 1000
        self.text_det_limit_type = "max"
        self.lang = lang


# ==================== OCR PROCESSOR ====================

class OCRProcessor:
    """Procesa OCR en imágenes/PDFs"""
    
    def __init__(self, config: OCRConfig):
        self.ocr = PaddleOCR(
            use_doc_orientation_classify=config.use_doc_orientation_classify,
            use_doc_unwarping=config.use_doc_unwarping,
            use_textline_orientation=config.use_textline_orientation,
            text_det_limit_side_len=config.text_det_limit_side_len,
            text_det_limit_type=config.text_det_limit_type,
            lang=config.lang,
        )
    
    def process(self, input_path: str):
        """Ejecuta OCR en el archivo de entrada"""
        return self.ocr.predict(input=input_path)


# ==================== DNI EXTRACTOR ====================

class DNIExtractor:
    """Responsable de extraer DNIs de los resultados OCR"""
    
    PATRON_DNI = r'\b(\d{8})\b'
    PATRON_DNI_EN_LINEA = r'\bDNI\b.*?(\d{8})\b'
    VENTANA_BUSQUEDA = 5
    
    @classmethod
    def extract_all_from_results(cls, ocr_result) -> List[DNIInfo]:
        """Extrae TODOS los DNIs posibles de los resultados OCR con priorización mejorada"""
        dnis_encontrados = []
        
        for res in ocr_result:
            textos = res['rec_texts']
            confianzas = res['rec_scores']
            page = res['page_index']

            # ✅ ESTRATEGIA 2 (PRIORIDAD MAXIMA): Buscar DNIs en la MISMA línea
            dnis_en_linea = cls._buscar_dnis_en_misma_linea(
                textos, confianzas, page
            )
            dnis_encontrados.extend(dnis_en_linea)

            # ✅ ESTRATEGIA 1 (PRIORIDAD MEDIA): Buscar DNIs con etiqueta "DNI"
            dnis_con_etiqueta = cls._buscar_dnis_con_etiqueta(
                textos, confianzas, page
            )

            # Agregar solo los que no estén ya en la lista
            numeros_existentes = {dni.numero for dni in dnis_encontrados}
            for dni in dnis_con_etiqueta:
                if dni.numero not in numeros_existentes:
                    dnis_encontrados.append(dni)
            
            # # ✅ ESTRATEGIA 3 (PRIORIDAD MENOR): Solo números de 8 dígitos
            # dnis_sin_etiqueta = cls._buscar_todos_dnis_sin_etiqueta(
            #     textos, confianzas, page
            # )
            
            # # Agregar solo los que no estén ya en la lista
            # for dni in dnis_sin_etiqueta:
            #     if dni.numero not in numeros_existentes:
            #         dnis_encontrados.append(dni)
            #         numeros_existentes.add(dni.numero)
        
        return dnis_encontrados
    
    @classmethod
    def _buscar_dnis_en_misma_linea(
        cls, 
        textos: List[str], 
        confianzas: List[float], 
        page: int
    ) -> List[DNIInfo]:
        """✅ NUEVA: Busca DNI y número en la MISMA línea (ej: 'DNI 78887021', 'DNI asd 78887021')"""
        dnis = []
        
        for j, texto in enumerate(textos):
            texto_upper = texto.upper()
            match = re.search(cls.PATRON_DNI_EN_LINEA, texto_upper)
            
            if match:
                dnis.append(DNIInfo(
                    numero=match.group(1),
                    confianza=confianzas[j],
                    pagina=page,
                    posicion=j + 1
                ))
        
        return dnis
    
    @classmethod
    def _buscar_dnis_con_etiqueta(
        cls, 
        textos: List[str], 
        confianzas: List[float], 
        page: int
    ) -> List[DNIInfo]:
        """Busca DNIs cerca de la etiqueta 'DNI' (líneas separadas)"""
        dnis = []
        for j, texto in enumerate(textos):
            if cls._es_etiqueta_dni(texto):
                # ✅ PRIMERO: Verificar si el DNI está en la MISMA línea
                texto_upper = texto.upper()
                match_en_linea = re.search(cls.PATRON_DNI_EN_LINEA, texto_upper)
                
                if match_en_linea:
                    # Ya se detectó en _buscar_dnis_en_misma_linea, saltar
                    continue
                
                # ✅ SEGUNDO: Buscar en las siguientes líneas
                dni = cls._buscar_dni_siguiente(textos, confianzas, j)
                if dni:
                    dnis.append(DNIInfo(
                        numero=dni['numero'],
                        confianza=dni['confianza'],
                        pagina=page,
                        posicion=dni['posicion']
                    ))
        return dnis
    
    @classmethod
    def _buscar_todos_dnis_sin_etiqueta(
        cls, 
        textos: List[str], 
        confianzas: List[float], 
        page: int
    ) -> List[DNIInfo]:
        """Busca TODOS los números de 8 dígitos (sin etiqueta)"""
        dnis = []
        for j, texto in enumerate(textos):
            match = re.search(cls.PATRON_DNI, texto)
            if match:
                dnis.append(DNIInfo(
                    numero=match.group(1),
                    confianza=confianzas[j],
                    pagina=page,
                    posicion=j + 1
                ))
        return dnis
    
    @classmethod
    def _es_etiqueta_dni(cls, texto: str) -> bool:
        """Verifica si el texto contiene la etiqueta DNI como palabra completa"""
        return bool(re.search(r'\bDNI\b', texto.upper()))
    
    @classmethod
    def _buscar_dni_siguiente(
        cls, 
        textos: List[str], 
        confianzas: List[float], 
        indice_actual: int
    ) -> Optional[Dict]:
        """Busca el DNI en las siguientes líneas (después de encontrar 'DNI')"""
        rango_fin = min(indice_actual + cls.VENTANA_BUSQUEDA, len(textos))
        
        for k in range(indice_actual, rango_fin):
            texto = textos[k]
            match = re.search(cls.PATRON_DNI, texto)
            if match:
                return {
                    'numero': match.group(1),
                    'confianza': confianzas[k],
                    'posicion': k + 1
                }
        return None


# ==================== WEB SCRAPER SERVICE ====================

class WebScraperConfig:
    """Configuración para el web scraper"""
    URL_BASE = 'https://eldni.com/'
    TIMEOUT = 10
    WAIT_TIME = 2
    WAIT_TIME_RESULT = 5


class WebScraperService:
    """Responsable de realizar web scraping para obtener información de DNI"""
    
    def __init__(self, config: WebScraperConfig = WebScraperConfig()):
        self.config = config
        self.chrome_options = self._configure_chrome_options()
    
    def _configure_chrome_options(self) -> Options:
        """Configura las opciones de Chrome"""
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        return options
    
    def buscar_nombre_por_dni(self, dni: str) -> Optional[str]:
        """Busca el nombre completo usando el DNI mediante web scraping"""
        driver = None
        try:
            driver = webdriver.Chrome(
                service=Service(ChromeDriverManager().install()), 
                options=self.chrome_options
            )
            return self._realizar_busqueda(driver, dni)
        except Exception as e:
            print(f"⚠ Error al buscar DNI {dni}: {str(e)[:50]}...")
            return None
        finally:
            if driver:
                driver.quit()
    
    def _realizar_busqueda(self, driver, dni: str) -> Optional[str]:
        """Realiza la búsqueda en la página web"""
        driver.get(self.config.URL_BASE)
        
        # Ingresar DNI
        input_dni = WebDriverWait(driver, self.config.TIMEOUT).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="dni"]'))
        )
        time.sleep(self.config.WAIT_TIME)
        input_dni.send_keys(dni)
        
        # Hacer clic en buscar
        boton_buscar = WebDriverWait(driver, self.config.TIMEOUT).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="btn-buscar-datos-por-dni"]'))
        )
        boton_buscar.click()
        
        # Obtener resultado
        time.sleep(self.config.WAIT_TIME_RESULT)
        texto_resultado = WebDriverWait(driver, self.config.TIMEOUT).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="column-center"]/div[1]/div[1]/samp'))
        )
        
        return texto_resultado.text


# ==================== DNI VALIDATOR ====================

class DNIValidator:
    """Valida DNIs usando web scraping"""
    
    def __init__(self, web_scraper: WebScraperService):
        self.web_scraper = web_scraper
    
    def validate_dnis(self, dnis: List[DNIInfo]) -> Optional[PersonaInfo]:
        """Valida cada DNI con web scraping hasta encontrar uno válido"""
        print('\n🔍 Validando DNIs con web scraping...\n')
        
        for idx, dni in enumerate(dnis, 1):
            print(f'Intento {idx}/{len(dnis)} - Probando DNI: {dni.numero}')
            nombre_completo = self.web_scraper.buscar_nombre_por_dni(dni.numero)
            
            if nombre_completo:
                print(f'✓ DNI VÁLIDO ENCONTRADO: {dni.numero}')
                return PersonaInfo(
                    dni=dni,
                    nombre_completo=nombre_completo
                )
            else:
                print(f'✗ DNI inválido o no encontrado: {dni.numero}')
        
        print('\n⚠ Ningún DNI fue válido en el web scraping')
        return None


# ==================== RESULT PRINTER ====================

class ResultPrinter:
    """Responsable de imprimir resultados"""
    
    @staticmethod
    def print_header(title: str):
        """Imprime un encabezado formateado"""
        print(f'\n{"="*50}')
        print(f'=== {title} ===')
        print(f'{"="*50}')
    
    @staticmethod
    def print_ocr_texts(ocr_result):
        """Imprime todos los textos detectados por OCR"""
        ResultPrinter.print_header("PALABRAS DETECTADAS")
        
        for res in ocr_result:
            textos = res['rec_texts']
            confianzas = res['rec_scores']
            page = res['page_index']
            
            for j, texto in enumerate(textos, 1):
                print(f'{j}. text: {texto} - confianza: {confianzas[j-1]:.4f} - página: {page}')
        
        print(f'{"="*50}')
    
    @staticmethod
    def print_dnis_found(dnis: List[DNIInfo]):
        """Imprime todos los DNIs encontrados"""
        ResultPrinter.print_header("DNIs ENCONTRADOS")
        
        if not dnis:
            print('⚠️  No se encontraron DNIs')
            print(f'{"="*50}')
            return
        
        for idx, dni in enumerate(dnis, 1):
            print(f'{idx}. DNI: {dni.numero} - Confianza: {dni.confianza:.4f} - '
                  f'Página: {dni.pagina} - Posición: {dni.posicion}')
        
        print(f'{"="*50}')
    
    @staticmethod
    def print_final_result(persona: Optional[PersonaInfo]):
        """Imprime el resultado final del procesamiento"""
        ResultPrinter.print_header("RESULTADO FINAL")
        
        if persona:
            print(f'✓ DNI: {persona.dni.numero}')
            print(f'✓ Confianza OCR: {persona.dni.confianza:.4f}')
            print(f'✓ Página: {persona.dni.pagina}')
            print(f'✓ NOMBRE COMPLETO: {persona.nombre_completo}')
        else:
            print('✗ No se encontró un DNI válido')
        
        print(f'{"="*50}')


# ==================== MAIN ORCHESTRATOR ====================

class ComprobanteDNIProcessor:
    """Orquestador principal para procesar DNIs en comprobantes"""
    
    def __init__(self, input_path: str):
        self.input_path = input_path
        self.ocr_processor = OCRProcessor(OCRConfig())
        self.dni_extractor = DNIExtractor()
        self.dni_validator = DNIValidator(WebScraperService())
        self.printer = ResultPrinter()
    
    def process(self) -> tuple[str, str]:  # ✅ Retorna (nombre_completo, dni)
        """Procesa el comprobante y retorna (nombre_completo, dni) o cadenas vacías"""
        # 1. Ejecutar OCR
        print("🔍 Procesando OCR...")
        ocr_result = self.ocr_processor.process(self.input_path)
        
        # 2. Mostrar textos detectados
        self.printer.print_ocr_texts(ocr_result)
        
        # 3. Extraer DNIs
        dnis_encontrados = self.dni_extractor.extract_all_from_results(ocr_result)
        
        # 4. Mostrar DNIs encontrados
        print(f'\n📋 Total de DNIs candidatos encontrados: {len(dnis_encontrados)}')
        self.printer.print_dnis_found(dnis_encontrados)
        
        # 5. Validar DNIs con web scraping
        if not dnis_encontrados:
            self.printer.print_final_result(None)
            return '', ''  # ✅ Retorna cadenas vacías si no hay DNIs
        
        persona_valida = self.dni_validator.validate_dnis(dnis_encontrados)
        
        # 6. Mostrar resultado final
        self.printer.print_final_result(persona_valida)
        
        # ✅ Retorna tupla con nombre completo y DNI, o cadenas vacías
        if persona_valida:
            nombre_completo = persona_valida.nombre_completo if persona_valida.nombre_completo else ''
            dni = persona_valida.dni.numero if persona_valida.dni else ''
            return nombre_completo, dni
        else:
            return '', ''


# ==================== EXECUTION ====================

if __name__ == '__main__':
    input_path = "comprobantes/images/93-hojas-conv-54_png.rf.58a0ff14fd3c0d552bdfa9c4fb38c06d.jpg"

    # "comprobante/comprobante con dni y ruc (SAN PABLO).pdf"
    # "comprobantes/images/08_jpg.rf.808bc0b8bc656f62f9ed4d4236135e9e.jpg"
    
    processor = ComprobanteDNIProcessor(input_path)
    nombre_completo, dni = processor.process()
    
    # ✅ Mostrar resultados
    print(f'\nNombre completo extraído: {nombre_completo}')
    print(f'DNI extraído: {dni}')

Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/franspaxi/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('latin_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/franspaxi/.paddlex/official_models/latin_PP-OCRv5_mobile_rec`.


🔍 Procesando OCR...

=== PALABRAS DETECTADAS ===
1. text: Intichay - confianza: 0.9299 - página: None
2. text: R.U.C. 20564510301 - confianza: 0.9978 - página: None
3. text: BOLETA DE VENTA - confianza: 0.9995 - página: None
4. text: Dirección Fiscal: - confianza: 0.9994 - página: None
5. text: AV.NICANOR LARREA S/N - SANTA ANA - LA CONVENCION - confianza: 0.9980 - página: None
6. text: - CUSCO - confianza: 0.9631 - página: None
7. text: ELECTRONICA - confianza: 1.0000 - página: None
8. text: PUNTO DE EMISION: AV.NICANOR LARREA S/N SANTA ANA LA CONVENCION - confianza: 0.9859 - página: None
9. text: B601 - confianza: 1.0000 - página: None
10. text: N° 00813084 - confianza: 0.9983 - página: None
11. text: CLIENTE - confianza: 0.9999 - página: None
12. text: : BACA LOYZA EDUARDA - confianza: 0.9912 - página: None
13. text: Dirección - confianza: 0.9996 - página: None
14. text: : PRLG.EDGAR DE LA TORRE SN SANTA ANA LA CONVENCION CUSCO - confianza: 0.9891 - página: None
15. text: CODIGO - c

## Clasificar el RUC para conseguir el NOMBRE COMERCIAL de la empresa

In [ ]:
from dataclasses import dataclass
from paddleocr import PaddleOCR
import re
import time
from typing import List, Optional, Dict
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


# ==================== DATA CLASSES ====================

@dataclass
class RUCInfo:
    """Información del RUC encontrado"""
    numero: str
    confianza: float
    pagina: int
    posicion: int


@dataclass
class EmpresaInfo:
    """Información completa de una empresa"""
    ruc: RUCInfo
    razon_social: Optional[str]


# ==================== OCR CONFIGURATION ====================

class OCRConfig:
    """Configuración para PaddleOCR"""
    
    def __init__(self, lang: str = 'es'):
        self.use_doc_orientation_classify = False
        self.use_doc_unwarping = False
        self.use_textline_orientation = False
        self.text_det_limit_side_len = 1000
        self.text_det_limit_type = "max"
        self.lang = lang


# ==================== OCR PROCESSOR ====================

class OCRProcessor:
    """Procesa OCR en imágenes/PDFs"""
    
    def __init__(self, config: OCRConfig):
        self.ocr = PaddleOCR(
            use_doc_orientation_classify=config.use_doc_orientation_classify,
            use_doc_unwarping=config.use_doc_unwarping,
            use_textline_orientation=config.use_textline_orientation,
            text_det_limit_side_len=config.text_det_limit_side_len,
            text_det_limit_type=config.text_det_limit_type,
            lang=config.lang,
        )
    
    def process(self, input_path: str):
        """Ejecuta OCR en el archivo de entrada"""
        return self.ocr.predict(input=input_path)


# ==================== RUC EXTRACTOR ====================

class RUCExtractor:
    """Responsable de extraer RUCs de los resultados OCR"""
    
    PATRON_RUC = r'\b(\d{11})\b'
    PATRON_RUC_EN_LINEA = r'\bR\.?U\.?C\.?\b.*?(\d{11})\b'  # ✅ MODIFICADO: Detecta "RUC", "R.U.C", "R.U.C."
    VENTANA_BUSQUEDA = 3
    
    @classmethod
    def extract_all_from_results(cls, ocr_result) -> List[RUCInfo]:
        """Extrae TODOS los RUCs posibles de los resultados OCR con priorización mejorada"""
        rucs_encontrados = []
        
        for res in ocr_result:
            textos = res['rec_texts']
            confianzas = res['rec_scores']
            page = res['page_index']

            # ✅ ESTRATEGIA 2 (PRIORIDAD MAXIMA): Buscar RUCs en la MISMA línea
            rucs_en_linea = cls._buscar_rucs_en_misma_linea(
                textos, confianzas, page
            )
            rucs_encontrados.extend(rucs_en_linea)

            # ✅ ESTRATEGIA 1 (PRIORIDAD MEDIA): Buscar RUCs con etiqueta "RUC"
            rucs_con_etiqueta = cls._buscar_rucs_con_etiqueta(
                textos, confianzas, page
            )

            # Agregar solo los que no estén ya en la lista
            numeros_existentes = {ruc.numero for ruc in rucs_encontrados}
            for ruc in rucs_con_etiqueta:
                if ruc.numero not in numeros_existentes:
                    rucs_encontrados.append(ruc)
            
            # # ✅ ESTRATEGIA 3 (MENOR PRIORIDAD): Solo números de 11 dígitos
            # rucs_sin_etiqueta = cls._buscar_todos_rucs_sin_etiqueta(
            #     textos, confianzas, page
            # )
            
            # # Agregar solo los que no estén ya en la lista
            # for ruc in rucs_sin_etiqueta:
            #     if ruc.numero not in numeros_existentes:
            #         rucs_encontrados.append(ruc)
            #         numeros_existentes.add(ruc.numero)
        
        return rucs_encontrados
    
    @classmethod
    def _buscar_rucs_en_misma_linea(
        cls, 
        textos: List[str], 
        confianzas: List[float], 
        page: int
    ) -> List[RUCInfo]:
        """✅ MODIFICADO: Busca RUC/R.U.C y número en la MISMA línea"""
        rucs = []
        
        for j, texto in enumerate(textos):
            texto_upper = texto.upper()
            match = re.search(cls.PATRON_RUC_EN_LINEA, texto_upper)
            
            if match:
                rucs.append(RUCInfo(
                    numero=match.group(1),
                    confianza=confianzas[j],
                    pagina=page,
                    posicion=j + 1
                ))
        
        return rucs
    
    @classmethod
    def _buscar_rucs_con_etiqueta(
        cls, 
        textos: List[str], 
        confianzas: List[float], 
        page: int
    ) -> List[RUCInfo]:
        """Busca RUCs cerca de la etiqueta 'RUC' o 'R.U.C' (líneas separadas)"""
        rucs = []
        for j, texto in enumerate(textos):
            if cls._es_etiqueta_ruc(texto):
                # ✅ PRIMERO: Verificar si el RUC está en la MISMA línea
                texto_upper = texto.upper()
                match_en_linea = re.search(cls.PATRON_RUC_EN_LINEA, texto_upper)
                
                if match_en_linea:
                    # Ya se detectó en _buscar_rucs_en_misma_linea, saltar
                    continue
                
                # ✅ SEGUNDO: Buscar en las siguientes líneas
                ruc = cls._buscar_ruc_siguiente(textos, confianzas, j)
                if ruc:
                    rucs.append(RUCInfo(
                        numero=ruc['numero'],
                        confianza=ruc['confianza'],
                        pagina=page,
                        posicion=ruc['posicion']
                    ))
        return rucs
    
    @classmethod
    def _buscar_todos_rucs_sin_etiqueta(
        cls, 
        textos: List[str], 
        confianzas: List[float], 
        page: int
    ) -> List[RUCInfo]:
        """Busca TODOS los números de 11 dígitos (sin etiqueta)"""
        rucs = []
        for j, texto in enumerate(textos):
            match = re.search(cls.PATRON_RUC, texto)
            if match:
                rucs.append(RUCInfo(
                    numero=match.group(1),
                    confianza=confianzas[j],
                    pagina=page,
                    posicion=j + 1
                ))
        return rucs
    
    @classmethod
    def _es_etiqueta_ruc(cls, texto: str) -> bool:
        """✅ MODIFICADO: Verifica si contiene 'RUC' o 'R.U.C' como palabra completa"""
        return bool(re.search(r'\bR\.?U\.?C\.?\b', texto.upper()))
    
    @classmethod
    def _buscar_ruc_siguiente(
        cls, 
        textos: List[str], 
        confianzas: List[float], 
        indice_actual: int
    ) -> Optional[Dict]:
        """Busca el RUC en las siguientes líneas (después de encontrar 'RUC')"""
        rango_fin = min(indice_actual + cls.VENTANA_BUSQUEDA, len(textos))
        
        for k in range(indice_actual, rango_fin):
            texto = textos[k]
            match = re.search(cls.PATRON_RUC, texto)
            if match:
                return {
                    'numero': match.group(1),
                    'confianza': confianzas[k],
                    'posicion': k + 1
                }
        return None


# ==================== WEB SCRAPER SERVICE ====================

class WebScraperConfigRUC:
    """Configuración para el web scraper de RUC"""
    URL_BASE = 'https://e-consultaruc.sunat.gob.pe/cl-ti-itmrconsruc/FrameCriterioBusquedaWeb.jsp'
    TIMEOUT = 10
    WAIT_TIME = 5
    MAX_REINTENTOS = 3
    REINTENTO_DELAY = 3


class WebScraperServiceRUC:
    """Responsable de realizar web scraping para obtener información de RUC"""
    
    def __init__(self, config: WebScraperConfigRUC = WebScraperConfigRUC()):
        self.config = config
        self.chrome_options = self._configure_chrome_options()
    
    def _configure_chrome_options(self) -> Options:
        """Configura las opciones de Chrome para evitar detección"""
        options = Options()
        options.add_argument('--headless=new')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-blink-features=AutomationControlled')
        options.add_argument('--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36')
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
        return options
    
    def buscar_razon_social_por_ruc(self, ruc: str) -> Optional[str]:
        """Busca la razón social con sistema de reintentos"""
        for intento in range(self.config.MAX_REINTENTOS):
            driver = None
            try:
                driver = webdriver.Chrome(
                    service=Service(ChromeDriverManager().install()), 
                    options=self.chrome_options
                )
                resultado = self._realizar_busqueda(driver, ruc)
                return resultado
            except Exception as e:
                print(f"⚠ Intento {intento + 1}/{self.config.MAX_REINTENTOS} falló: {str(e)[:50]}...")
                if intento < self.config.MAX_REINTENTOS - 1:
                    time.sleep(self.config.REINTENTO_DELAY)
            finally:
                if driver:
                    driver.quit()
        
        return None
    
    def _realizar_busqueda(self, driver, ruc: str) -> Optional[str]:
        """Realiza la búsqueda en la página web de SUNAT"""
        driver.get(self.config.URL_BASE)
        
        # Ingresar RUC
        input_ruc = WebDriverWait(driver, self.config.TIMEOUT).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="txtRuc"]'))
        )
        time.sleep(self.config.WAIT_TIME)
        input_ruc.send_keys(ruc)
        
        # Hacer clic en buscar
        boton_buscar = WebDriverWait(driver, self.config.TIMEOUT).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="btnAceptar"]'))
        )
        boton_buscar.click()
        
        # Obtener resultado (segundo h4 - Razón Social)
        time.sleep(self.config.WAIT_TIME)
        elementos_h4 = driver.find_elements(By.CSS_SELECTOR, "div.list-group h4")
        
        if len(elementos_h4) >= 2:
            texto_completo = elementos_h4[1].text  # "20601725551 - CLINICA CERRO COLORADO S.A.C."
            
            # ✅ EXTRAER SOLO LA RAZÓN SOCIAL (después del " - ")
            if ' - ' in texto_completo:
                razon_social = texto_completo.split(' - ', 1)[1]  # "CLINICA CERRO COLORADO S.A.C."
                return razon_social
            else:
                # Si no hay " - ", retornar el texto completo
                return texto_completo
        
        return None


# ==================== RUC VALIDATOR ====================

class RUCValidator:
    """Valida RUCs usando web scraping"""
    
    def __init__(self, web_scraper: WebScraperServiceRUC):
        self.web_scraper = web_scraper
    
    def validate_rucs(self, rucs: List[RUCInfo]) -> Optional[EmpresaInfo]:
        """Valida cada RUC con web scraping hasta encontrar uno válido"""
        print('\n🔍 Validando RUCs con web scraping...\n')
        
        for idx, ruc in enumerate(rucs, 1):
            print(f'Intento {idx}/{len(rucs)} - Probando RUC: {ruc.numero}')
            razon_social = self.web_scraper.buscar_razon_social_por_ruc(ruc.numero)
            
            if razon_social:
                print(f'✓ RUC VÁLIDO ENCONTRADO: {ruc.numero}')
                return EmpresaInfo(
                    ruc=ruc,
                    razon_social=razon_social
                )
            else:
                print(f'✗ RUC inválido o no encontrado: {ruc.numero}')
        
        print('\n⚠ Ningún RUC fue válido en el web scraping')
        return None


# ==================== RESULT PRINTER ====================

class ResultPrinter:
    """Responsable de imprimir resultados"""
    
    @staticmethod
    def print_header(title: str):
        """Imprime un encabezado formateado"""
        print(f'\n{"="*50}')
        print(f'=== {title} ===')
        print(f'{"="*50}')
    
    @staticmethod
    def print_ocr_texts(ocr_result):
        """Imprime todos los textos detectados por OCR"""
        ResultPrinter.print_header("PALABRAS DETECTADAS")
        
        for res in ocr_result:
            textos = res['rec_texts']
            confianzas = res['rec_scores']
            page = res['page_index']
            
            for j, texto in enumerate(textos, 1):
                print(f'{j}. text: {texto} - confianza: {confianzas[j-1]:.4f} - página: {page}')
        
        print(f'{"="*50}')
    
    @staticmethod
    def print_rucs_found(rucs: List[RUCInfo]):
        """Imprime todos los RUCs encontrados"""
        ResultPrinter.print_header("RUCs ENCONTRADOS")
        
        if not rucs:
            print('⚠️  No se encontraron RUCs')
            print(f'{"="*50}')
            return
        
        for idx, ruc in enumerate(rucs, 1):
            print(f'{idx}. RUC: {ruc.numero} - Confianza: {ruc.confianza:.4f} - '
                  f'Página: {ruc.pagina} - Posición: {ruc.posicion}')
        
        print(f'{"="*50}')
    
    @staticmethod
    def print_final_result(empresa: Optional[EmpresaInfo]):
        """Imprime el resultado final del procesamiento"""
        ResultPrinter.print_header("RESULTADO FINAL")
        
        if empresa:
            print(f'✓ RUC: {empresa.ruc.numero}')
            print(f'✓ Confianza OCR: {empresa.ruc.confianza:.4f}')
            print(f'✓ Página: {empresa.ruc.pagina}')
            print(f'✓ RAZÓN SOCIAL: {empresa.razon_social}')
        else:
            print('✗ No se encontró un RUC válido')
        
        print(f'{"="*50}')


# ==================== MAIN ORCHESTRATOR ====================

class ComprobanteRUCProcessor:
    """Orquestador principal para procesar RUCs en comprobantes"""
    
    def __init__(self, input_path: str):
        self.input_path = input_path
        self.ocr_processor = OCRProcessor(OCRConfig())
        self.ruc_extractor = RUCExtractor()
        self.ruc_validator = RUCValidator(WebScraperServiceRUC())
        self.printer = ResultPrinter()
    
    def process(self) -> tuple[str, str]:  # ✅ Cambiado: retorna (razon_social, ruc)
        """Procesa el comprobante completo y retorna (razón_social, ruc) o cadenas vacías"""
        # 1. Ejecutar OCR
        print("🔍 Procesando OCR...")
        ocr_result = self.ocr_processor.process(self.input_path)
        
        # 2. Mostrar textos detectados
        self.printer.print_ocr_texts(ocr_result)
        
        # 3. Extraer RUCs
        rucs_encontrados = self.ruc_extractor.extract_all_from_results(ocr_result)
        
        # 4. Mostrar RUCs encontrados
        print(f'\n📋 Total de RUCs candidatos encontrados: {len(rucs_encontrados)}')
        self.printer.print_rucs_found(rucs_encontrados)
        
        # 5. Validar RUCs con web scraping
        if not rucs_encontrados:
            self.printer.print_final_result(None)
            return '', ''  # ✅ Retorna cadenas vacías si no hay RUCs
        
        empresa_valida = self.ruc_validator.validate_rucs(rucs_encontrados)
        
        # 6. Mostrar resultado final
        self.printer.print_final_result(empresa_valida)
        
        # ✅ Retorna tupla con razón social y RUC, o cadenas vacías
        if empresa_valida:
            razon_social = empresa_valida.razon_social if empresa_valida.razon_social else ''
            ruc = empresa_valida.ruc.numero if empresa_valida.ruc else ''
            return razon_social, ruc
        else:
            return '', ''


# ==================== EXECUTION ====================

if __name__ == '__main__':
    input_path = "comprobante/comprobante sin nombre de cliente (NUBEFACT).png"
    
    processor = ComprobanteRUCProcessor(input_path)
    razon_social, ruc = processor.process()
    
    # ✅ Mostrar resultados
    print(f'\nRazón social extraída: {razon_social}')
    print(f'RUC extraído: {ruc}')

Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/franspaxi/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('latin_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/franspaxi/.paddlex/official_models/latin_PP-OCRv5_mobile_rec`.


🔍 Procesando OCR...

=== PALABRAS DETECTADAS ===
1. text: NubeFacT - confianza: 0.9937 - página: None
2. text: RUC 20600695771 - confianza: 0.9999 - página: None
3. text: Validación OSE - confianza: 0.9711 - página: None
4. text: FACTURA ELECTRÓNICA - confianza: 0.9860 - página: None
5. text:  - confianza: 0.0000 - página: None
6. text: FFF1-228790 - confianza: 1.0000 - página: None
7. text: NUBEFACT SA - confianza: 0.9998 - página: None
8. text: CALLE LIBERTAD 176 OF 303 MIRAFLORES LIMA DEPARTAMENTO LIMA - confianza: 0.9957 - página: None
9. text: Validación de XML según SUNAT - confianza: 0.9681 - página: None
10. text: CLIENTE: - confianza: 0.9999 - página: None
11. text: FECHA EMISIÓN: 11/09/2025 - confianza: 0.9841 - página: None
12. text: RUC: 20145496170 - confianza: 0.9693 - página: None
13. text: HORA EMISIÓN: 16:31:19 - confianza: 0.9994 - página: None
14. text: DENOMINACIÓN: UNIVERSIDAD NACIONAL DEL ALTIPLANO PUNO - confianza: 0.9882 - página: None
15. text: FECHA DE VENC.: 

## Clasificar el TIPO DE COMPROBANTE 

In [ ]:
from dataclasses import dataclass
from paddleocr import PaddleOCR
import re
from typing import List, Optional

# ==================== DATA CLASSES ====================

@dataclass
class TipoComprobanteInfo:
    """Información del tipo de comprobante detectado"""
    tipo: str
    confianza: float
    texto_original: str
    pagina: int
    posicion: int


# ==================== OCR PROCESSOR ====================

class OCRConfig:
    """Configuración para PaddleOCR"""
    
    def __init__(self, lang: str = 'es'):
        self.use_doc_orientation_classify = False
        self.use_doc_unwarping = False
        self.use_textline_orientation = False
        self.text_det_limit_side_len = 1000
        self.text_det_limit_type = "max"
        self.lang = lang


class OCRProcessor:
    """Procesa OCR en imágenes/PDFs"""
    
    def __init__(self, config: OCRConfig):
        self.ocr = PaddleOCR(
            use_doc_orientation_classify=config.use_doc_orientation_classify,
            use_doc_unwarping=config.use_doc_unwarping,
            use_textline_orientation=config.use_textline_orientation,
            text_det_limit_side_len=config.text_det_limit_side_len,
            text_det_limit_type=config.text_det_limit_type,
            lang=config.lang,
        )
    
    def process(self, input_path: str):
        """Ejecuta OCR en el archivo de entrada"""
        return self.ocr.predict(input=input_path)


# ==================== TIPO COMPROBANTE EXTRACTOR ====================

class TipoComprobanteExtractor:
    """Responsable de extraer el tipo de comprobante de los resultados OCR"""
    
    # Patrones de tipos de comprobantes (ordenados por prioridad)
    TIPOS_COMPROBANTE = {
        'FACTURA ELECTRÓNICA': r'(?<!\w)FACTURA\s*ELECTR[OÓ]NICA(?!\w)',
        'BOLETA DE VENTA ELECTRÓNICA': r'(?<!\w)BOLETA\s*(?:DE\s*VENTA\s*)?ELECTR[OÓ]NICA(?!\w)',
        'NOTA DE CRÉDITO ELECTRÓNICA': r'(?<!\w)NOTA\s*DE\s*CR[EÉ]DITO\s*ELECTR[OÓ]NICA(?!\w)',
        'NOTA DE DÉBITO ELECTRÓNICA': r'(?<!\w)NOTA\s*DE\s*D[EÉ]BITO\s*ELECTR[OÓ]NICA(?!\w)',
        'RECIBO POR HONORARIOS ELECTRÓNICO': r'(?<!\w)RECIBO\s*(?:POR\s*)?HONORARIOS\s*ELECTR[OÓ]NIC[OA](?!\w)',
        'FACTURA': r'(?<!\w)FACTURA(?!\s*ELECTR[OÓ]NICA)(?!\w)',
        'BOLETA DE VENTA': r'(?<!\w)BOLETA\s*(?:DE\s*VENTA)?(?!\s*ELECTR[OÓ]NICA)(?!\w)',
        'TICKET': r'(?<!\w)(?:TICKET|TIKKET)(?!\w)',
    }
    
    @classmethod
    def extract_from_results(cls, ocr_result) -> Optional[TipoComprobanteInfo]:
        """Extrae el tipo de comprobante de los resultados OCR"""
        candidatos = []
        
        for res in ocr_result:
            textos = res['rec_texts']
            confianzas = res['rec_scores']
            page = res['page_index']
            
            for j, texto in enumerate(textos):
                tipo_detectado = cls._detectar_tipo(texto)
                if tipo_detectado:
                    candidatos.append(TipoComprobanteInfo(
                        tipo=tipo_detectado,
                        confianza=confianzas[j],
                        texto_original=texto,
                        pagina=page,
                        posicion=j + 1
                    ))
        
        # ✅ Retornar el candidato con MAYOR CONFIANZA y MENOR POSICIÓN
        if candidatos:
            # Priorizar por confianza, luego por posición (aparición más temprana)
            return max(candidatos, key=lambda x: (x.confianza, -x.posicion))
        
        return None
    
    @classmethod
    def _detectar_tipo(cls, texto: str) -> Optional[str]:
        """Detecta el tipo de comprobante en un texto usando patrones regex"""
        texto_upper = texto.upper()
        
        # ✅ Intentar primero con patrones ESPECÍFICOS
        for tipo, patron in cls.TIPOS_COMPROBANTE.items():
            if re.search(patron, texto_upper):
                return tipo
        
        return None


# ==================== RESULT PRINTER ====================

class ResultPrinter:
    """Responsable de imprimir resultados"""
    
    @staticmethod
    def print_header(title: str):
        """Imprime un encabezado formateado"""
        print(f'\n{"="*50}')
        print(f'=== {title} ===')
        print(f'{"="*50}')
    
    @staticmethod
    def print_ocr_texts(ocr_result):
        """Imprime todos los textos detectados por OCR"""
        ResultPrinter.print_header("PALABRAS DETECTADAS")
        
        for res in ocr_result:
            textos = res['rec_texts']
            confianzas = res['rec_scores']
            page = res['page_index']
            
            for j, texto in enumerate(textos, 1):
                print(f'{j}. text: {texto} - confianza: {confianzas[j-1]:.4f} - página: {page}')
        
        print(f'{"="*50}')
    
    @staticmethod
    def print_tipo_comprobante(tipo_info: Optional[TipoComprobanteInfo]):
        """Imprime el tipo de comprobante detectado"""
        ResultPrinter.print_header("TIPO DE COMPROBANTE DETECTADO")
        
        if tipo_info:
            print(f'📄 TIPO: {tipo_info.tipo}')
            print(f'📄 Texto Original: "{tipo_info.texto_original}"')
            print(f'📄 Confianza: {tipo_info.confianza:.4f}')
            print(f'📄 Página: {tipo_info.pagina}')
            print(f'📄 Posición: {tipo_info.posicion}')
        else:
            print('⚠️  No se detectó el tipo de comprobante')
        
        print(f'{"="*50}')


# ==================== MAIN ORCHESTRATOR ====================

class ComprobanteTipoProcessor:
    """Orquestador principal para procesar el tipo de comprobante"""
    
    def __init__(self, input_path: str):
        self.input_path = input_path
        self.ocr_processor = OCRProcessor(OCRConfig())
        self.tipo_extractor = TipoComprobanteExtractor()
        self.printer = ResultPrinter()
    
    def process(self) -> str:
        """Procesa el comprobante y retorna el tipo detectado o cadena vacía"""
        # 1. Ejecutar OCR
        print("🔍 Procesando OCR...")
        ocr_result = self.ocr_processor.process(self.input_path)
        
        # 2. Mostrar textos detectados
        self.printer.print_ocr_texts(ocr_result)
        
        # 3. Extraer tipo de comprobante
        tipo_info = self.tipo_extractor.extract_from_results(ocr_result)
        
        # 4. Mostrar tipo detectado
        self.printer.print_tipo_comprobante(tipo_info)
        
        # 5. Retornar tipo o cadena vacía
        if tipo_info:
            return tipo_info.tipo
        else:
            return ''


# ==================== EXECUTION ====================

if __name__ == '__main__':
    input_path = "comprobante/comprobante con dni y ruc (SAN PABLO).pdf"
    
    processor = ComprobanteTipoProcessor(input_path)
    tipo_comprobante = processor.process()
    
    print(f'\nTipo de comprobante extraído: {tipo_comprobante}')

Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/franspaxi/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('latin_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/franspaxi/.paddlex/official_models/latin_PP-OCRv5_mobile_rec`.


🔍 Procesando OCR...

=== PALABRAS DETECTADAS ===
1. text: Clínica - confianza: 1.0000 - página: 0
2. text: San Pablo - confianza: 0.9618 - página: 0
3. text: Arequipa - confianza: 0.9999 - página: 0
4. text: Cinica Cerro Colorado SAC - RUC: 20601725551 - confianza: 0.9808 - página: 0
5. text: URB. SANTA TERESA MZ. K LT 9 CERRO COLORADO-AREQUIPA - confianza: 0.9829 - página: 0
6. text: (CITY CENTER - AL FRENTE DE LAS TORRES QUIMERA) - confianza: 0.9955 - página: 0
7. text: Telf: +51(54)410100 - confianza: 0.9991 - página: 0
8. text: BOLETA DE VENTA ELECTRÓNICA - confianza: 0.9995 - página: 0
9. text: B041 - 00010015 - confianza: 0.9962 - página: 0
10. text: PAXI JUCHANI FRANS EDWARD - confianza: 0.9888 - página: 0
11. text: DNI 78887021 - confianza: 0.9992 - página: 0
12. text: FECHA DE EMISION: - confianza: 0.9983 - página: 0
13. text: 2025-07-16 - confianza: 1.0000 - página: 0
14. text: HORA: - confianza: 0.9995 - página: 0
15. text: 11:23:51 - confianza: 0.9782 - página: 0
16. text: 

## Clasificar la FECHA DE EMISION del comprobante

In [ ]:
from dataclasses import dataclass
from paddleocr import PaddleOCR
import re
from typing import List, Optional
from datetime import datetime

# ==================== DATA CLASSES ====================

@dataclass
class FechaEmisionInfo:
    """Información de la fecha de emisión encontrada"""
    fecha_original: str
    fecha_normalizada: str  # Formato YYYY-MM-DD
    confianza: float
    pagina: int
    posicion: int


# ==================== OCR PROCESSOR ====================

class OCRConfig:
    """Configuración para PaddleOCR"""
    
    def __init__(self, lang: str = 'es'):
        self.use_doc_orientation_classify = False
        self.use_doc_unwarping = False
        self.use_textline_orientation = False
        self.text_det_limit_side_len = 1000
        self.text_det_limit_type = "max"
        self.lang = lang


class OCRProcessor:
    """Procesa OCR en imágenes/PDFs"""
    
    def __init__(self, config: OCRConfig):
        self.ocr = PaddleOCR(
            use_doc_orientation_classify=config.use_doc_orientation_classify,
            use_doc_unwarping=config.use_doc_unwarping,
            use_textline_orientation=config.use_textline_orientation,
            text_det_limit_side_len=config.text_det_limit_side_len,
            text_det_limit_type=config.text_det_limit_type,
            lang=config.lang,
        )
    
    def process(self, input_path: str):
        """Ejecuta OCR en el archivo de entrada"""
        return self.ocr.predict(input=input_path)


# ==================== FECHA EXTRACTOR ====================

class FechaExtractor:
    """Responsable de extraer fechas de emisión de los resultados OCR"""
    
    # Patrones de fechas (ordenados por prioridad)
    PATRONES_FECHA = [
        # Formato ISO: 2025-07-16
        (r'(\d{4})-(\d{2})-(\d{2})', '%Y-%m-%d'),
        
        # Formato DD/MM/YYYY: 20/02/2022
        (r'(\d{2})/(\d{2})/(\d{4})', '%d/%m/%Y'),
        
        # Formato DD-MM-YYYY: 20-02-2022
        (r'(\d{2})-(\d{2})-(\d{4})', '%d-%m-%Y'),
        
        # Formato DD.MM.YYYY: 20.02.2022
        (r'(\d{2})\.(\d{2})\.(\d{4})', '%d.%m.%Y'),
    ]
    
    # ✅ NUEVO: Patrón para detectar FECHA + NÚMERO en la misma línea
    PATRON_FECHA_EN_LINEA = r'(?:FECHA\s*(?:DE\s*)?(?:EMISI[OÓ]N)?|F\.\s*EMISI[OÓ]N|FEC\.\s*EMISI[OÓ]N|FECHA:).*?(\d{2}[/-]\d{2}[/-]\d{4}|\d{4}-\d{2}-\d{2})'
    
    # Palabras clave que indican fecha de emisión
    KEYWORDS_FECHA_EMISION = [
        'FECHA EMISIÓN',
        'FECHA EMISION',
        'FECHA DE EMISIÓN',
        'FECHA DE EMISION',
        'FECHA:',
        'F. EMISIÓN',
        'F. EMISION',
        'FEC. EMISIÓN',
        'FEC. EMISION',
    ]
    
    VENTANA_BUSQUEDA = 3
    
    @classmethod
    def extract_from_results(cls, ocr_result) -> Optional[FechaEmisionInfo]:
        """Extrae la fecha de emisión de los resultados OCR con priorización mejorada"""
        candidatos = []
        
        for res in ocr_result:
            textos = res['rec_texts']
            confianzas = res['rec_scores']
            page = res['page_index']
            
            # ✅ ESTRATEGIA 1 (PRIORIDAD MAXIMA): Buscar fechas en la MISMA línea que el keyword
            fechas_en_linea = cls._buscar_fechas_en_misma_linea(
                textos, confianzas, page
            )
            candidatos.extend(fechas_en_linea)

            # ✅ ESTRATEGIA 2 (PRIORIDAD MEDIA): Buscar fechas con etiqueta "FECHA EMISIÓN" (líneas separadas)
            fechas_con_keyword = cls._buscar_fechas_con_keyword(
                textos, confianzas, page
            )
            
            # Agregar solo las que no estén ya en la lista
            fechas_existentes = {(fecha.fecha_normalizada, fecha.pagina) for fecha in candidatos}
            for fecha in fechas_con_keyword:
                if (fecha.fecha_normalizada, fecha.pagina) not in fechas_existentes:
                    candidatos.append(fecha)
            
            # # ✅ ESTRATEGIA 3 (PRIORIDAD BAJA): Buscar TODAS las fechas sin etiqueta
            # fechas_sin_keyword = cls._buscar_todas_fechas_sin_keyword(
            #     textos, confianzas, page
            # )
            
            # # Agregar solo las que no estén ya en la lista
            # for fecha in fechas_sin_keyword:
            #     if (fecha.fecha_normalizada, fecha.pagina) not in fechas_existentes:
            #         candidatos.append(fecha)
            #         fechas_existentes.add((fecha.fecha_normalizada, fecha.pagina))

        # Retornar el candidato con mayor confianza y menor posición
        if candidatos:
            return max(candidatos, key=lambda x: (x.confianza, -x.posicion))
        
        return None
    
    @classmethod
    def _buscar_fechas_con_keyword(
        cls, 
        textos: List[str], 
        confianzas: List[float], 
        page: int
    ) -> List[FechaEmisionInfo]:
        """✅ PRIORIDAD 1: Busca fechas cerca de palabras clave (líneas separadas)"""
        fechas = []
        
        for j, texto in enumerate(textos):
            if cls._contiene_keyword_fecha(texto):
                # ✅ PRIMERO: Verificar si la fecha está en la MISMA línea
                texto_upper = texto.upper()
                match_en_linea = re.search(cls.PATRON_FECHA_EN_LINEA, texto_upper, re.IGNORECASE)
                
                if match_en_linea:
                    # Ya se detectó en _buscar_fechas_en_misma_linea, saltar
                    continue
                
                # ✅ SEGUNDO: Buscar en las siguientes líneas
                fecha = cls._buscar_fecha_siguiente(textos, confianzas, j, page)
                if fecha:
                    fechas.append(fecha)
        
        return fechas
    
    @classmethod
    def _buscar_fechas_en_misma_linea(
        cls, 
        textos: List[str], 
        confianzas: List[float], 
        page: int
    ) -> List[FechaEmisionInfo]:
        """✅ PRIORIDAD 2: Busca fechas en la MISMA línea que el keyword"""
        fechas = []
        
        for j, texto in enumerate(textos):
            texto_upper = texto.upper()
            
            # ✅ Buscar patrón "FECHA EMISIÓN: 20/02/2022" o "FECHA: 20/02/2022"
            match = re.search(cls.PATRON_FECHA_EN_LINEA, texto_upper, re.IGNORECASE)
            
            if match:
                fecha_str = match.group(1)
                
                # Intentar extraer fecha del string encontrado
                fecha_info = cls._extraer_fecha_de_texto(
                    fecha_str, 
                    confianzas[j], 
                    page, 
                    j + 1
                )
                if fecha_info:
                    fechas.append(fecha_info)
        
        return fechas
    
    @classmethod
    def _buscar_todas_fechas_sin_keyword(
        cls, 
        textos: List[str], 
        confianzas: List[float], 
        page: int
    ) -> List[FechaEmisionInfo]:
        """✅ PRIORIDAD 3: Busca TODAS las fechas sin etiqueta"""
        fechas = []
        
        for j, texto in enumerate(textos):
            # Solo buscar si NO contiene el keyword (ya se procesaron antes)
            if not cls._contiene_keyword_fecha(texto):
                fecha_info = cls._extraer_fecha_de_texto(texto, confianzas[j], page, j + 1)
                if fecha_info:
                    fechas.append(fecha_info)
        
        return fechas
    
    @classmethod
    def _buscar_fecha_siguiente(
        cls, 
        textos: List[str], 
        confianzas: List[float], 
        indice_actual: int,
        page: int
    ) -> Optional[FechaEmisionInfo]:
        """Busca una fecha en las siguientes líneas (después de encontrar keyword)"""
        rango_fin = min(indice_actual + cls.VENTANA_BUSQUEDA, len(textos))
        
        for k in range(indice_actual, rango_fin):
            texto = textos[k]
            fecha_info = cls._extraer_fecha_de_texto(texto, confianzas[k], page, k + 1)
            if fecha_info:
                return fecha_info
        
        return None
    
    @classmethod
    def _extraer_fecha_de_texto(
        cls, 
        texto: str, 
        confianza: float, 
        page: int, 
        posicion: int
    ) -> Optional[FechaEmisionInfo]:
        """Extrae una fecha de un texto usando múltiples patrones"""
        for patron, formato in cls.PATRONES_FECHA:
            match = re.search(patron, texto)
            if match:
                fecha_original = match.group(0)
                
                # Intentar normalizar la fecha
                fecha_normalizada = cls._normalizar_fecha(fecha_original, formato)
                
                if fecha_normalizada:
                    return FechaEmisionInfo(
                        fecha_original=fecha_original,
                        fecha_normalizada=fecha_normalizada,
                        confianza=confianza,
                        pagina=page,
                        posicion=posicion
                    )
        
        return None
    
    @classmethod
    def _normalizar_fecha(cls, fecha_str: str, formato: str) -> Optional[str]:
        """Normaliza una fecha al formato YYYY-MM-DD"""
        try:
            # Intentar parsear con el formato dado
            fecha_obj = datetime.strptime(fecha_str, formato)
            # Retornar en formato ISO
            return fecha_obj.strftime('%Y-%m-%d')
        except ValueError:
            # Si el formato DD/MM/YYYY falla, intentar con MM/DD/YYYY
            if formato == '%d/%m/%Y':
                try:
                    fecha_obj = datetime.strptime(fecha_str, '%m/%d/%Y')
                    return fecha_obj.strftime('%Y-%m-%d')
                except ValueError:
                    pass
            return None
    
    @classmethod
    def _contiene_keyword_fecha(cls, texto: str) -> bool:
        """Verifica si el texto contiene alguna palabra clave de fecha"""
        texto_upper = texto.upper()
        return any(keyword in texto_upper for keyword in cls.KEYWORDS_FECHA_EMISION)


# ==================== RESULT PRINTER ====================

class ResultPrinter:
    """Responsable de imprimir resultados"""
    
    @staticmethod
    def print_header(title: str):
        """Imprime un encabezado formateado"""
        print(f'\n{"="*50}')
        print(f'=== {title} ===')
        print(f'{"="*50}')
    
    @staticmethod
    def print_ocr_texts(ocr_result):
        """Imprime todos los textos detectados por OCR"""
        ResultPrinter.print_header("PALABRAS DETECTADAS")
        
        for res in ocr_result:
            textos = res['rec_texts']
            confianzas = res['rec_scores']
            page = res['page_index']
            
            for j, texto in enumerate(textos, 1):
                print(f'{j}. text: {texto} - confianza: {confianzas[j-1]:.4f} - página: {page}')
        
        print(f'{"="*50}')
    
    @staticmethod
    def print_fecha_emision(fecha_info: Optional[FechaEmisionInfo]):
        """Imprime la fecha de emisión detectada"""
        ResultPrinter.print_header("FECHA DE EMISIÓN DETECTADA")
        
        if fecha_info:
            print(f'📅 FECHA ORIGINAL: {fecha_info.fecha_original}')
            print(f'📅 FECHA NORMALIZADA: {fecha_info.fecha_normalizada}')
            print(f'📅 Confianza: {fecha_info.confianza:.4f}')
            print(f'📅 Página: {fecha_info.pagina}')
            print(f'📅 Posición: {fecha_info.posicion}')
        else:
            print('⚠️  No se detectó la fecha de emisión')
        
        print(f'{"="*50}')


# ==================== MAIN ORCHESTRATOR ====================

class ComprobanteFechaProcessor:
    """Orquestador principal para procesar la fecha de emisión"""
    
    def __init__(self, input_path: str):
        self.input_path = input_path
        self.ocr_processor = OCRProcessor(OCRConfig())
        self.fecha_extractor = FechaExtractor()
        self.printer = ResultPrinter()
    
    def process(self) -> str:
        """Procesa el comprobante y retorna la fecha normalizada o cadena vacía"""
        # 1. Ejecutar OCR
        print("🔍 Procesando OCR...")
        ocr_result = self.ocr_processor.process(self.input_path)
        
        # 2. Mostrar textos detectados
        self.printer.print_ocr_texts(ocr_result)
        
        # 3. Extraer fecha de emisión
        fecha_info = self.fecha_extractor.extract_from_results(ocr_result)
        
        # 4. Mostrar fecha detectada
        self.printer.print_fecha_emision(fecha_info)
        
        # 5. Retornar fecha normalizada o cadena vacía
        if fecha_info:
            return fecha_info.fecha_normalizada
        else:
            return ''


# ==================== EXECUTION ====================

if __name__ == '__main__':
    input_path = "comprobante/comprobante sin nombre de cliente 2 (MIFARMA).jpeg"
    
    processor = ComprobanteFechaProcessor(input_path)
    fecha_emision = processor.process()
    
    print(f'\nFecha de emisión extraída: {fecha_emision}')

Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/franspaxi/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('latin_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/franspaxi/.paddlex/official_models/latin_PP-OCRv5_mobile_rec`.


🔍 Procesando OCR...

=== PALABRAS DETECTADAS ===
1. text: MIFARMA S.A.C. - RUC: 20512002090 - confianza: 0.9858 - página: None
2. text: CENTRAL: Cal. Victor Alzamora Nro. 147 Urb. Santa - confianza: 0.9664 - página: None
3. text: Catalina - confianza: 0.9996 - página: None
4. text: La Victoria - Lima TELf.: 2130760 - confianza: 0.9833 - página: None
5. text: TIENDA C55 - JULIACA SAN ROMAN 3 - 1052 - confianza: 0.9808 - página: None
6. text: Jr. San Roman Nro. 521 Puno - San Roman - Juliaca - confianza: 0.9835 - página: None
7. text: BOLETA DE VENTA ELECTRONICA BC55-00408563 - confianza: 0.9968 - página: None
8. text: FECHA EMISION: 10/09/2025 12:47:02 NP:0001062881 - confianza: 0.9932 - página: None
9. text: CAJA/TURNO: 51/1 - confianza: 0.9753 - página: None
10. text: CAJERO: NQ***PE - confianza: 0.9637 - página: None
11. text: CODIGODESCRIPCION - confianza: 0.9996 - página: None
12. text: CANT. P.UNIT. DSCTO. IMPORTE - confianza: 0.9768 - página: None
13. text: 571264 RINOFLUIMUCIL S

## Clasificacion de Comprobante COMPLETO

In [9]:
from dataclasses import dataclass
from paddleocr import PaddleOCR
import re
import time
from typing import List, Optional, Dict
from datetime import datetime
from abc import ABC, abstractmethod
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


# ==================== DATA CLASSES ====================

@dataclass
class ComprobanteResponse:
    """Respuesta del procesamiento del comprobante"""
    nombre_cliente: str
    dni_o_ruc_cliente: str
    tipo_comprobante: str
    fecha_emision: str
    ruc_empresa: str
    nombre_empresa: str
    importe_total: str

@dataclass
class NumeroDetectado:
    """Información de un número decimal detectado"""
    texto_original: str
    valor_numerico: float
    confianza: float
    pagina: int
    posicion: int


@dataclass
class DNIInfo:
    """Información del DNI encontrado"""
    numero: str
    confianza: float
    pagina: int
    posicion: int


@dataclass
class PersonaInfo:
    """Información completa de una persona"""
    dni: DNIInfo
    nombre_completo: Optional[str]


@dataclass
class RUCInfo:
    """Información del RUC encontrado"""
    numero: str
    confianza: float
    pagina: int
    posicion: int


@dataclass
class EmpresaInfo:
    """Información completa de una empresa"""
    ruc: RUCInfo
    razon_social: Optional[str]


@dataclass
class TipoComprobanteInfo:
    """Información del tipo de comprobante detectado"""
    tipo: str
    confianza: float
    texto_original: str
    pagina: int
    posicion: int


@dataclass
class FechaEmisionInfo:
    """Información de la fecha de emisión encontrada"""
    fecha_original: str
    fecha_normalizada: str
    confianza: float
    pagina: int
    posicion: int


# ==================== OCR CONFIGURATION ====================

class OCRConfig:
    """Configuración para PaddleOCR"""
    
    def __init__(self, lang: str = 'es'):
        self.use_doc_orientation_classify = False
        self.use_doc_unwarping = False
        self.use_textline_orientation = False
        self.text_det_limit_side_len = 1000
        self.text_det_limit_type = "max"
        self.lang = lang


# ==================== OCR PROCESSOR ====================

class OCRProcessor:
    """Procesa OCR en imágenes/PDFs"""
    
    def __init__(self, config: OCRConfig):
        self.ocr = PaddleOCR(
            use_doc_orientation_classify=config.use_doc_orientation_classify,
            use_doc_unwarping=config.use_doc_unwarping,
            use_textline_orientation=config.use_textline_orientation,
            text_det_limit_side_len=config.text_det_limit_side_len,
            text_det_limit_type=config.text_det_limit_type,
            lang=config.lang,
        )
    
    def process(self, input_path: str):
        """Ejecuta OCR en el archivo de entrada"""
        return self.ocr.predict(input=input_path)


# ==================== NUMBER PATTERNS & EXTRACTORS ====================

class NumberPattern(ABC):
    """Clase base abstracta para patrones de números"""
    
    @abstractmethod
    def match(self, texto: str) -> Optional[float]:
        """Intenta extraer un número del texto"""
        pass


# ✅ NUEVO PATRÓN: Para "S 77.23," o "S 77.23" (sin barra, con/sin coma final)
class SimpleDecimalWithSPattern(NumberPattern):
    """Patrón para formato S 77.23, o S 77.23 (sin barra, con coma opcional al final)"""
    
    PATTERN = r'^S\s*(\d+\.\d{2}),?$'  # ✅ Detecta "S 77.23," o "S 77.23"
    
    def match(self, texto: str) -> Optional[float]:
        match = re.match(self.PATTERN, texto)
        if match:
            return float(match.group(1))
        return None


class ComaSeparadorMilesPattern(NumberPattern):
    """Patrón para formato 1,298.45 (peruano/USA) con S/ opcional"""
    
    PATTERN = r'^(?:S/\s*)?[\$\s]*(\d{1,3}(?:,\d{3})*\.\d{2})$'
    
    def match(self, texto: str) -> Optional[float]:
        match = re.match(self.PATTERN, texto)
        if match:
            numero_str = match.group(1).replace(',', '')
            return float(numero_str)
        return None


class PuntoSeparadorMilesPattern(NumberPattern):
    """Patrón para formato europeo 1.298,45 con S/ opcional"""
    
    PATTERN = r'^(?:S/\s*)?[\$\s]*(\d{1,3}(?:\.\d{3})*,\d{2})$'
    
    def match(self, texto: str) -> Optional[float]:
        match = re.match(self.PATTERN, texto)
        if match:
            numero_str = match.group(1).replace('.', '').replace(',', '.')
            return float(numero_str)
        return None


class SimpleDecimalPattern(NumberPattern):
    """Patrón para formato simple 150.00 con S/ opcional"""
    
    PATTERN = r'^(?:S/\s*)?[\$\s]*(\d+\.\d{2})$'
    
    def match(self, texto: str) -> Optional[float]:
        match = re.match(self.PATTERN, texto)
        if match:
            return float(match.group(1))
        return None


class EnteroConSeparadorPattern(NumberPattern):
    """Patrón para números enteros con separador 1,500 con S/ opcional - SOLO CON DECIMALES .00"""
    
    # ✅ MODIFICADO: Ahora requiere .00 al final (números enteros con decimales explícitos)
    PATTERN = r'^(?:S/\s*)?[\$\s]*(\d{1,3}(?:,\d{3})*)\.\d{2}$'
    
    def match(self, texto: str) -> Optional[float]:
        match = re.match(self.PATTERN, texto)
        if match:
            numero_str = match.group(1).replace(',', '')
            # Verificar si el texto tiene parte decimal
            if '.' in texto:
                # Extraer el número completo incluyendo decimales
                numero_completo = texto.replace('S/', '').replace('$', '').replace(',', '').strip()
                return float(numero_completo)
            return float(numero_str)
        return None


# ✅ NUEVO: Patrón para números con decimales (prioridad alta)
class DecimalExplicitoPattern(NumberPattern):
    """Patrón para números con decimales explícitos: 19.96, 1,298.45"""
    
    PATTERN = r'^(?:S/\s*)?[\$\s]*(\d{1,3}(?:,\d{3})*\.\d{2})$'
    
    def match(self, texto: str) -> Optional[float]:
        match = re.match(self.PATTERN, texto)
        if match:
            numero_str = match.group(1).replace(',', '')
            return float(numero_str)
        return None


class DecimalExtractor:
    """Extractor de números decimales usando múltiples patrones"""
    
    def __init__(self):
        self.patterns: List[NumberPattern] = [
            SimpleDecimalWithSPattern(),      # "S 77.23,"
            DecimalExplicitoPattern(),        # ✅ NUEVO: Prioriza "19.96", "1,298.45"
            ComaSeparadorMilesPattern(),      # "1,298.45"
            PuntoSeparadorMilesPattern(),     # "1.298,45" (europeo)
            SimpleDecimalPattern(),           # "150.00"
            # ✅ REMOVIDO: EnteroConSeparadorPattern() ya no captura "305" sin decimales
        ]
    
    def extract(self, texto: str) -> Optional[float]:
        """Intenta extraer un número decimal del texto usando todos los patrones"""
        texto = texto.strip()
        
        for pattern in self.patterns:
            resultado = pattern.match(texto)
            if resultado is not None:
                return resultado
        
        return None


# ==================== NUMBER ANALYZER ====================

class NumberAnalyzer:
    """Analiza y extrae números de resultados OCR"""
    
    def __init__(self, extractor: DecimalExtractor):
        self.extractor = extractor
    
    def analyze(self, ocr_result) -> List[NumeroDetectado]:
        """Analiza resultados OCR y extrae todos los números CON DECIMALES"""
        numeros_detectados = []
        
        for res in ocr_result:
            textos = res['rec_texts']
            confianzas = res['rec_scores']
            page = res['page_index']
            
            for j, texto in enumerate(textos):
                numero = self.extractor.extract(texto)
                
                # ✅ NUEVO: Solo agregar si tiene parte decimal diferente de .00
                # O si el texto original contiene un punto decimal
                if numero is not None:
                    # Verificar si es un número con decimales significativos
                    parte_decimal = numero - int(numero)
                    
                    # Solo agregar si:
                    # 1. Tiene decimales diferentes de .00 (ej: 19.96)
                    # 2. O el texto original contiene "." (para capturar casos como "39.92")
                    if parte_decimal > 0 or '.' in texto:
                        numeros_detectados.append(NumeroDetectado(
                            texto_original=texto,
                            valor_numerico=numero,
                            confianza=confianzas[j],
                            pagina=page,
                            posicion=j + 1
                        ))
        
        return numeros_detectados


# ==================== DNI EXTRACTOR ====================

class DNIExtractor:
    """Responsable de extraer DNIs de los resultados OCR"""
    
    PATRON_DNI = r'\b(\d{8})\b'
    PATRON_DNI_EN_LINEA = r'\bDNI\b.*?(\d{8})\b'
    VENTANA_BUSQUEDA = 3
    
    @classmethod
    def extract_all_from_results(cls, ocr_result) -> List[DNIInfo]:
        """Extrae TODOS los DNIs posibles de los resultados OCR con priorización mejorada"""
        dnis_encontrados = []
        
        for res in ocr_result:
            textos = res['rec_texts']
            confianzas = res['rec_scores']
            page = res['page_index']
            
            # ✅ ESTRATEGIA 1 (PRIORIDAD MAXIMA): Buscar DNIs en la MISMA línea
            dnis_en_linea = cls._buscar_dnis_en_misma_linea(
                textos, confianzas, page
            )
            dnis_encontrados.extend(dnis_en_linea)

            # ✅ ESTRATEGIA 2 (PRIORIDAD MEDIA): Buscar DNIs con etiqueta "DNI"
            dnis_con_etiqueta = cls._buscar_dnis_con_etiqueta(
                textos, confianzas, page
            )

            # Agregar solo los que no estén ya en la lista
            numeros_existentes = {dni.numero for dni in dnis_encontrados}
            for dni in dnis_con_etiqueta:
                if dni.numero not in numeros_existentes:
                    dnis_encontrados.append(dni)
        
        return dnis_encontrados
    
    @classmethod
    def _buscar_dnis_en_misma_linea(
        cls, 
        textos: List[str], 
        confianzas: List[float], 
        page: int
    ) -> List[DNIInfo]:
        """✅ NUEVA: Busca DNI y número en la MISMA línea (ej: 'DNI 78887021', 'DNI asd 78887021')"""
        dnis = []
        
        for j, texto in enumerate(textos):
            texto_upper = texto.upper()
            match = re.search(cls.PATRON_DNI_EN_LINEA, texto_upper)
            
            if match:
                dnis.append(DNIInfo(
                    numero=match.group(1),
                    confianza=confianzas[j],
                    pagina=page,
                    posicion=j + 1
                ))
        
        return dnis
    
    @classmethod
    def _buscar_dnis_con_etiqueta(
        cls, 
        textos: List[str], 
        confianzas: List[float], 
        page: int
    ) -> List[DNIInfo]:
        """Busca DNIs cerca de la etiqueta 'DNI' (líneas separadas)"""
        dnis = []
        for j, texto in enumerate(textos):
            if cls._es_etiqueta_dni(texto):
                # ✅ PRIMERO: Verificar si el DNI está en la MISMA línea
                texto_upper = texto.upper()
                match_en_linea = re.search(cls.PATRON_DNI_EN_LINEA, texto_upper)
                
                if match_en_linea:
                    # Ya se detectó en _buscar_dnis_en_misma_linea, saltar
                    continue
                
                # ✅ SEGUNDO: Buscar en las siguientes líneas
                dni = cls._buscar_dni_siguiente(textos, confianzas, j)
                if dni:
                    dnis.append(DNIInfo(
                        numero=dni['numero'],
                        confianza=dni['confianza'],
                        pagina=page,
                        posicion=dni['posicion']
                    ))
        return dnis
    
    @classmethod
    def _buscar_todos_dnis_sin_etiqueta(
        cls, 
        textos: List[str], 
        confianzas: List[float], 
        page: int
    ) -> List[DNIInfo]:
        """Busca TODOS los números de 8 dígitos (sin etiqueta)"""
        dnis = []
        for j, texto in enumerate(textos):
            match = re.search(cls.PATRON_DNI, texto)
            if match:
                dnis.append(DNIInfo(
                    numero=match.group(1),
                    confianza=confianzas[j],
                    pagina=page,
                    posicion=j + 1
                ))
        return dnis
    
    @classmethod
    def _es_etiqueta_dni(cls, texto: str) -> bool:
        """Verifica si el texto contiene la etiqueta DNI como palabra completa"""
        return bool(re.search(r'\bDNI\b', texto.upper()))
    
    @classmethod
    def _buscar_dni_siguiente(
        cls, 
        textos: List[str], 
        confianzas: List[float], 
        indice_actual: int
    ) -> Optional[Dict]:
        """Busca el DNI en las siguientes líneas (después de encontrar 'DNI')"""
        rango_fin = min(indice_actual + cls.VENTANA_BUSQUEDA, len(textos))
        
        for k in range(indice_actual, rango_fin):
            texto = textos[k]
            match = re.search(cls.PATRON_DNI, texto)
            if match:
                return {
                    'numero': match.group(1),
                    'confianza': confianzas[k],
                    'posicion': k + 1
                }
        return None


# ==================== RUC EXTRACTOR ====================

class RUCExtractor:
    """Responsable de extraer RUCs de los resultados OCR"""
    
    PATRON_RUC = r'\b(\d{11})\b'
    PATRON_RUC_EN_LINEA = r'\bR\.?U\.?C\.?\b.*?(\d{11})\b'  # ✅ MODIFICADO: Detecta "RUC", "R.U.C", "R.U.C."
    VENTANA_BUSQUEDA = 3
    
    @classmethod
    def extract_all_from_results(cls, ocr_result) -> List[RUCInfo]:
        """Extrae TODOS los RUCs posibles de los resultados OCR con priorización mejorada"""
        rucs_encontrados = []
        
        for res in ocr_result:
            textos = res['rec_texts']
            confianzas = res['rec_scores']
            page = res['page_index']

            # ✅ ESTRATEGIA 1 (PRIORIDAD MAXIMA): Buscar RUCs en la MISMA línea
            rucs_en_linea = cls._buscar_rucs_en_misma_linea(
                textos, confianzas, page
            )
            rucs_encontrados.extend(rucs_en_linea)

            # ✅ ESTRATEGIA 2 (PRIORIDAD MEDIA): Buscar RUCs con etiqueta "RUC"
            rucs_con_etiqueta = cls._buscar_rucs_con_etiqueta(
                textos, confianzas, page
            )

            # Agregar solo los que no estén ya en la lista
            numeros_existentes = {ruc.numero for ruc in rucs_encontrados}
            for ruc in rucs_con_etiqueta:
                if ruc.numero not in numeros_existentes:
                    rucs_encontrados.append(ruc)
            
            # # ✅ ESTRATEGIA 3 (MENOR PRIORIDAD): Solo números de 11 dígitos
            # rucs_sin_etiqueta = cls._buscar_todos_rucs_sin_etiqueta(
            #     textos, confianzas, page
            # )
            
            # # Agregar solo los que no estén ya en la lista
            # for ruc in rucs_sin_etiqueta:
            #     if ruc.numero not in numeros_existentes:
            #         rucs_encontrados.append(ruc)
            #         numeros_existentes.add(ruc.numero)
        
        return rucs_encontrados
    
    @classmethod
    def _buscar_rucs_en_misma_linea(
        cls, 
        textos: List[str], 
        confianzas: List[float], 
        page: int
    ) -> List[RUCInfo]:
        """✅ MODIFICADO: Busca RUC/R.U.C y número en la MISMA línea"""
        rucs = []
        
        for j, texto in enumerate(textos):
            texto_upper = texto.upper()
            match = re.search(cls.PATRON_RUC_EN_LINEA, texto_upper)
            
            if match:
                rucs.append(RUCInfo(
                    numero=match.group(1),
                    confianza=confianzas[j],
                    pagina=page,
                    posicion=j + 1
                ))
        
        return rucs
    
    @classmethod
    def _buscar_rucs_con_etiqueta(
        cls, 
        textos: List[str], 
        confianzas: List[float], 
        page: int
    ) -> List[RUCInfo]:
        """Busca RUCs cerca de la etiqueta 'RUC' o 'R.U.C' (líneas separadas)"""
        rucs = []
        for j, texto in enumerate(textos):
            if cls._es_etiqueta_ruc(texto):
                # ✅ PRIMERO: Verificar si el RUC está en la MISMA línea
                texto_upper = texto.upper()
                match_en_linea = re.search(cls.PATRON_RUC_EN_LINEA, texto_upper)
                
                if match_en_linea:
                    # Ya se detectó en _buscar_rucs_en_misma_linea, saltar
                    continue
                
                # ✅ SEGUNDO: Buscar en las siguientes líneas
                ruc = cls._buscar_ruc_siguiente(textos, confianzas, j)
                if ruc:
                    rucs.append(RUCInfo(
                        numero=ruc['numero'],
                        confianza=ruc['confianza'],
                        pagina=page,
                        posicion=ruc['posicion']
                    ))
        return rucs
    
    @classmethod
    def _buscar_todos_rucs_sin_etiqueta(
        cls, 
        textos: List[str], 
        confianzas: List[float], 
        page: int
    ) -> List[RUCInfo]:
        """Busca TODOS los números de 11 dígitos (sin etiqueta)"""
        rucs = []
        for j, texto in enumerate(textos):
            match = re.search(cls.PATRON_RUC, texto)
            if match:
                rucs.append(RUCInfo(
                    numero=match.group(1),
                    confianza=confianzas[j],
                    pagina=page,
                    posicion=j + 1
                ))
        return rucs
    
    @classmethod
    def _es_etiqueta_ruc(cls, texto: str) -> bool:
        """✅ MODIFICADO: Verifica si contiene 'RUC' o 'R.U.C' como palabra completa"""
        return bool(re.search(r'\bR\.?U\.?C\.?\b', texto.upper()))
    
    @classmethod
    def _buscar_ruc_siguiente(
        cls, 
        textos: List[str], 
        confianzas: List[float], 
        indice_actual: int
    ) -> Optional[Dict]:
        """Busca el RUC en las siguientes líneas (después de encontrar 'RUC')"""
        rango_fin = min(indice_actual + cls.VENTANA_BUSQUEDA, len(textos))
        
        for k in range(indice_actual, rango_fin):
            texto = textos[k]
            match = re.search(cls.PATRON_RUC, texto)
            if match:
                return {
                    'numero': match.group(1),
                    'confianza': confianzas[k],
                    'posicion': k + 1
                }
        return None
    

# ==================== TIPO COMPROBANTE EXTRACTOR ====================

class TipoComprobanteExtractor:
    """Responsable de extraer el tipo de comprobante de los resultados OCR"""
    
    TIPOS_COMPROBANTE = {
        'FACTURA ELECTRÓNICA': r'(?<!\w)FACTURA\s*ELECTR[OÓ]NICA(?!\w)',
        'BOLETA DE VENTA ELECTRÓNICA': r'(?<!\w)BOLETA\s*(?:DE\s*VENTA\s*)?ELECTR[OÓ]NICA(?!\w)',
        'NOTA DE CRÉDITO ELECTRÓNICA': r'(?<!\w)NOTA\s*DE\s*CR[EÉ]DITO\s*ELECTR[OÓ]NICA(?!\w)',
        'NOTA DE DÉBITO ELECTRÓNICA': r'(?<!\w)NOTA\s*DE\s*D[EÉ]BITO\s*ELECTR[OÓ]NICA(?!\w)',
        'RECIBO POR HONORARIOS ELECTRÓNICO': r'(?<!\w)RECIBO\s*(?:POR\s*)?HONORARIOS\s*ELECTR[OÓ]NIC[OA](?!\w)',
        'FACTURA': r'(?<!\w)FACTURA(?!\s*ELECTR[OÓ]NICA)(?!\w)',
        'BOLETA DE VENTA': r'(?<!\w)BOLETA\s*(?:DE\s*VENTA)?(?!\s*ELECTR[OÓ]NICA)(?!\w)',
        'TICKET': r'(?<!\w)(?:TICKET|TIKKET)(?!\w)',
    }
    
    @classmethod
    def extract_from_results(cls, ocr_result) -> Optional[TipoComprobanteInfo]:
        """Extrae el tipo de comprobante de los resultados OCR"""
        candidatos = []
        
        for res in ocr_result:
            textos = res['rec_texts']
            confianzas = res['rec_scores']
            page = res['page_index']
            
            for j, texto in enumerate(textos):
                tipo_detectado = cls._detectar_tipo(texto)
                if tipo_detectado:
                    candidatos.append(TipoComprobanteInfo(
                        tipo=tipo_detectado,
                        confianza=confianzas[j],
                        texto_original=texto,
                        pagina=page,
                        posicion=j + 1
                    ))
        
        if candidatos:
            return max(candidatos, key=lambda x: (x.confianza, -x.posicion))
        
        return None
    
    @classmethod
    def _detectar_tipo(cls, texto: str) -> Optional[str]:
        """Detecta el tipo de comprobante en un texto usando patrones regex"""
        texto_upper = texto.upper()
        
        for tipo, patron in cls.TIPOS_COMPROBANTE.items():
            if re.search(patron, texto_upper):
                return tipo
        
        return None


# ==================== FECHA EXTRACTOR ====================

class FechaExtractor:
    """Responsable de extraer fechas de emisión de los resultados OCR"""
    
    # Patrones de fechas (ordenados por prioridad)
    PATRONES_FECHA = [
        # Formato ISO: 2025-07-16
        (r'(\d{4})-(\d{2})-(\d{2})', '%Y-%m-%d'),
        
        # Formato DD/MM/YYYY: 20/02/2022
        (r'(\d{2})/(\d{2})/(\d{4})', '%d/%m/%Y'),
        
        # Formato DD-MM-YYYY: 20-02-2022
        (r'(\d{2})-(\d{2})-(\d{4})', '%d-%m-%Y'),
        
        # Formato DD.MM.YYYY: 20.02.2022
        (r'(\d{2})\.(\d{2})\.(\d{4})', '%d.%m.%Y'),
    ]
    
    # ✅ NUEVO: Patrón para detectar FECHA + NÚMERO en la misma línea
    PATRON_FECHA_EN_LINEA = r'(?:FECHA\s*(?:DE\s*)?(?:EMISI[OÓ]N)?|F\.\s*EMISI[OÓ]N|FEC\.\s*EMISI[OÓ]N|FECHA:).*?(\d{2}[/-]\d{2}[/-]\d{4}|\d{4}-\d{2}-\d{2})'
    
    # Palabras clave que indican fecha de emisión
    KEYWORDS_FECHA_EMISION = [
        'FECHA EMISIÓN',
        'FECHA EMISION',
        'FECHA DE EMISIÓN',
        'FECHA DE EMISION',
        'FECHA:',
        'F. EMISIÓN',
        'F. EMISION',
        'FEC. EMISIÓN',
        'FEC. EMISION',
    ]
    
    VENTANA_BUSQUEDA = 3
    
    @classmethod
    def extract_from_results(cls, ocr_result) -> Optional[FechaEmisionInfo]:
        """Extrae la fecha de emisión de los resultados OCR con priorización mejorada"""
        candidatos = []
        
        for res in ocr_result:
            textos = res['rec_texts']
            confianzas = res['rec_scores']
            page = res['page_index']

            # ✅ ESTRATEGIA 1 (PRIORIDAD MÁXIMA): Buscar fechas en la MISMA línea que el keyword
            fechas_en_linea = cls._buscar_fechas_en_misma_linea(
                textos, confianzas, page
            )
            candidatos.extend(fechas_en_linea)

            # ✅ ESTRATEGIA 2 (PRIORIDAD MEDIA): Buscar fechas con etiqueta "FECHA EMISIÓN" (líneas separadas)
            fechas_con_keyword = cls._buscar_fechas_con_keyword(
                textos, confianzas, page
            )
            
            # Agregar solo las que no estén ya en la lista
            fechas_existentes = {(fecha.fecha_normalizada, fecha.pagina) for fecha in candidatos}
            for fecha in fechas_con_keyword:
                if (fecha.fecha_normalizada, fecha.pagina) not in fechas_existentes:
                    candidatos.append(fecha)
            
            # # ✅ ESTRATEGIA 3 (PRIORIDAD BAJA): Buscar TODAS las fechas sin etiqueta
            # fechas_sin_keyword = cls._buscar_todas_fechas_sin_keyword(
            #     textos, confianzas, page
            # )
            
            # # Agregar solo las que no estén ya en la lista
            # for fecha in fechas_sin_keyword:
            #     if (fecha.fecha_normalizada, fecha.pagina) not in fechas_existentes:
            #         candidatos.append(fecha)
            #         fechas_existentes.add((fecha.fecha_normalizada, fecha.pagina))

        # Retornar el candidato con mayor confianza y menor posición
        if candidatos:
            return max(candidatos, key=lambda x: (x.confianza, -x.posicion))
            
        return None
    
    @classmethod
    def _buscar_fechas_con_keyword(
        cls, 
        textos: List[str], 
        confianzas: List[float], 
        page: int
    ) -> List[FechaEmisionInfo]:
        """✅ PRIORIDAD 1: Busca fechas cerca de palabras clave (líneas separadas)"""
        fechas = []
        
        for j, texto in enumerate(textos):
            if cls._contiene_keyword_fecha(texto):
                # ✅ PRIMERO: Verificar si la fecha está en la MISMA línea
                texto_upper = texto.upper()
                match_en_linea = re.search(cls.PATRON_FECHA_EN_LINEA, texto_upper, re.IGNORECASE)
                
                if match_en_linea:
                    # Ya se detectó en _buscar_fechas_en_misma_linea, saltar
                    continue
                
                # ✅ SEGUNDO: Buscar en las siguientes líneas
                fecha = cls._buscar_fecha_siguiente(textos, confianzas, j, page)
                if fecha:
                    fechas.append(fecha)
        
        return fechas
    
    @classmethod
    def _buscar_fechas_en_misma_linea(
        cls, 
        textos: List[str], 
        confianzas: List[float], 
        page: int
    ) -> List[FechaEmisionInfo]:
        """✅ PRIORIDAD 2: Busca fechas en la MISMA línea que el keyword"""
        fechas = []
        
        for j, texto in enumerate(textos):
            texto_upper = texto.upper()
            
            # ✅ Buscar patrón "FECHA EMISIÓN: 20/02/2022" o "FECHA: 20/02/2022"
            match = re.search(cls.PATRON_FECHA_EN_LINEA, texto_upper, re.IGNORECASE)
            
            if match:
                fecha_str = match.group(1)
                
                # Intentar extraer fecha del string encontrado
                fecha_info = cls._extraer_fecha_de_texto(
                    fecha_str, 
                    confianzas[j], 
                    page, 
                    j + 1
                )
                if fecha_info:
                    fechas.append(fecha_info)
        
        return fechas
    
    @classmethod
    def _buscar_todas_fechas_sin_keyword(
        cls, 
        textos: List[str], 
        confianzas: List[float], 
        page: int
    ) -> List[FechaEmisionInfo]:
        """✅ PRIORIDAD 3: Busca TODAS las fechas sin etiqueta"""
        fechas = []
        
        for j, texto in enumerate(textos):
            # Solo buscar si NO contiene el keyword (ya se procesaron antes)
            if not cls._contiene_keyword_fecha(texto):
                fecha_info = cls._extraer_fecha_de_texto(texto, confianzas[j], page, j + 1)
                if fecha_info:
                    fechas.append(fecha_info)
        
        return fechas
    
    @classmethod
    def _buscar_fecha_siguiente(
        cls, 
        textos: List[str], 
        confianzas: List[float], 
        indice_actual: int,
        page: int
    ) -> Optional[FechaEmisionInfo]:
        """Busca una fecha en las siguientes líneas (después de encontrar keyword)"""
        rango_fin = min(indice_actual + cls.VENTANA_BUSQUEDA, len(textos))
        
        for k in range(indice_actual, rango_fin):
            texto = textos[k]
            fecha_info = cls._extraer_fecha_de_texto(texto, confianzas[k], page, k + 1)
            if fecha_info:
                return fecha_info
        
        return None
    
    @classmethod
    def _extraer_fecha_de_texto(
        cls, 
        texto: str, 
        confianza: float, 
        page: int, 
        posicion: int
    ) -> Optional[FechaEmisionInfo]:
        """Extrae una fecha de un texto usando múltiples patrones"""
        for patron, formato in cls.PATRONES_FECHA:
            match = re.search(patron, texto)
            if match:
                fecha_original = match.group(0)
                
                # Intentar normalizar la fecha
                fecha_normalizada = cls._normalizar_fecha(fecha_original, formato)
                
                if fecha_normalizada:
                    return FechaEmisionInfo(
                        fecha_original=fecha_original,
                        fecha_normalizada=fecha_normalizada,
                        confianza=confianza,
                        pagina=page,
                        posicion=posicion
                    )
        
        return None
    
    @classmethod
    def _normalizar_fecha(cls, fecha_str: str, formato: str) -> Optional[str]:
        """Normaliza una fecha al formato YYYY-MM-DD"""
        try:
            # Intentar parsear con el formato dado
            fecha_obj = datetime.strptime(fecha_str, formato)
            # Retornar en formato ISO
            return fecha_obj.strftime('%Y-%m-%d')
        except ValueError:
            # Si el formato DD/MM/YYYY falla, intentar con MM/DD/YYYY
            if formato == '%d/%m/%Y':
                try:
                    fecha_obj = datetime.strptime(fecha_str, '%m/%d/%Y')
                    return fecha_obj.strftime('%Y-%m-%d')
                except ValueError:
                    pass
            return None
    
    @classmethod
    def _contiene_keyword_fecha(cls, texto: str) -> bool:
        """Verifica si el texto contiene alguna palabra clave de fecha"""
        texto_upper = texto.upper()
        return any(keyword in texto_upper for keyword in cls.KEYWORDS_FECHA_EMISION)


# ==================== WEB SCRAPER SERVICES ====================

class WebScraperConfig:
    """Configuración para el web scraper de DNI"""
    URL_BASE = 'https://eldni.com/'
    TIMEOUT = 5
    WAIT_TIME = 2
    WAIT_TIME_RESULT = 2


class WebScraperService:
    """Responsable de realizar web scraping para obtener información de DNI"""
    
    def __init__(self, config: WebScraperConfig = WebScraperConfig()):
        self.config = config
        self.chrome_options = self._configure_chrome_options()
    
    def _configure_chrome_options(self) -> Options:
        """Configura las opciones de Chrome"""
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        return options
    
    def buscar_nombre_por_dni(self, dni: str) -> Optional[str]:
        """Busca el nombre completo usando el DNI mediante web scraping"""
        driver = None
        try:
            driver = webdriver.Chrome(
                service=Service(ChromeDriverManager().install()), 
                options=self.chrome_options
            )
            return self._realizar_busqueda(driver, dni)
        except Exception as e:
            print(f"⚠ Error al buscar DNI {dni}: {str(e)[:50]}...")
            return None
        finally:
            if driver:
                driver.quit()
    
    def _realizar_busqueda(self, driver, dni: str) -> Optional[str]:
        """Realiza la búsqueda en la página web"""
        driver.get(self.config.URL_BASE)
        
        input_dni = WebDriverWait(driver, self.config.TIMEOUT).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="dni"]'))
        )
        time.sleep(self.config.WAIT_TIME)
        input_dni.send_keys(dni)
        
        boton_buscar = WebDriverWait(driver, self.config.TIMEOUT).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="btn-buscar-datos-por-dni"]'))
        )
        boton_buscar.click()
        
        time.sleep(self.config.WAIT_TIME_RESULT)
        texto_resultado = WebDriverWait(driver, self.config.TIMEOUT).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="column-center"]/div[1]/div[1]/samp'))
        )
        
        return texto_resultado.text


class WebScraperConfigRUC:
    """Configuración para el web scraper de RUC"""
    URL_BASE = 'https://e-consultaruc.sunat.gob.pe/cl-ti-itmrconsruc/FrameCriterioBusquedaWeb.jsp'
    TIMEOUT = 5
    WAIT_TIME = 2
    MAX_REINTENTOS = 2
    REINTENTO_DELAY = 3


class WebScraperServiceRUC:
    """Responsable de realizar web scraping para obtener información de RUC"""
    
    def __init__(self, config: WebScraperConfigRUC = WebScraperConfigRUC()):
        self.config = config
        self.chrome_options = self._configure_chrome_options()
    
    def _configure_chrome_options(self) -> Options:
        """Configura las opciones de Chrome para evitar detección"""
        options = Options()
        options.add_argument('--headless=new')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-blink-features=AutomationControlled')
        options.add_argument('--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36')
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
        return options
    
    def buscar_razon_social_por_ruc(self, ruc: str) -> Optional[str]:
        """Busca la razón social con sistema de reintentos"""
        for intento in range(self.config.MAX_REINTENTOS):
            driver = None
            try:
                driver = webdriver.Chrome(
                    service=Service(ChromeDriverManager().install()), 
                    options=self.chrome_options
                )
                resultado = self._realizar_busqueda(driver, ruc)
                return resultado
            except Exception as e:
                print(f"⚠ Intento {intento + 1}/{self.config.MAX_REINTENTOS} falló: {str(e)[:50]}...")
                if intento < self.config.MAX_REINTENTOS - 1:
                    time.sleep(self.config.REINTENTO_DELAY)
            finally:
                if driver:
                    driver.quit()
        
        return None
    
    def _realizar_busqueda(self, driver, ruc: str) -> Optional[str]:
        """Realiza la búsqueda en la página web de SUNAT"""
        driver.get(self.config.URL_BASE)
        
        input_ruc = WebDriverWait(driver, self.config.TIMEOUT).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="txtRuc"]'))
        )
        time.sleep(self.config.WAIT_TIME)
        input_ruc.send_keys(ruc)
        
        boton_buscar = WebDriverWait(driver, self.config.TIMEOUT).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="btnAceptar"]'))
        )
        boton_buscar.click()
        
        time.sleep(self.config.WAIT_TIME)
        elementos_h4 = driver.find_elements(By.CSS_SELECTOR, "div.list-group h4")
        
        if len(elementos_h4) >= 2:
            texto_completo = elementos_h4[1].text
            
            if ' - ' in texto_completo:
                razon_social = texto_completo.split(' - ', 1)[1]
                return razon_social
            else:
                return texto_completo
        
        return None


# ==================== VALIDATORS ====================

class DNIValidator:
    """Valida DNIs usando web scraping"""
    
    def __init__(self, web_scraper: WebScraperService):
        self.web_scraper = web_scraper
    
    def validate_dnis(self, dnis: List[DNIInfo]) -> Optional[PersonaInfo]:
        """Valida cada DNI con web scraping hasta encontrar uno válido"""
        print('\n🔍 Validando DNIs con web scraping...\n')
        
        for idx, dni in enumerate(dnis, 1):
            print(f'Intento {idx}/{len(dnis)} - Probando DNI: {dni.numero}')
            nombre_completo = self.web_scraper.buscar_nombre_por_dni(dni.numero)
            
            if nombre_completo:
                print(f'✓ DNI VÁLIDO ENCONTRADO: {dni.numero}')
                return PersonaInfo(dni=dni, nombre_completo=nombre_completo)
            else:
                print(f'✗ DNI inválido o no encontrado: {dni.numero}')
        
        print('\n⚠ Ningún DNI fue válido en el web scraping')
        return None


class RUCValidator:
    """Valida RUCs usando web scraping"""
    
    def __init__(self, web_scraper: WebScraperServiceRUC):
        self.web_scraper = web_scraper
    
    def validate_rucs(self, rucs: List[RUCInfo]) -> Optional[EmpresaInfo]:
        """Valida cada RUC con web scraping hasta encontrar uno válido"""
        print('\n🔍 Validando RUCs con web scraping...\n')
        
        for idx, ruc in enumerate(rucs, 1):
            print(f'Intento {idx}/{len(rucs)} - Probando RUC: {ruc.numero}')
            razon_social = self.web_scraper.buscar_razon_social_por_ruc(ruc.numero)
            
            if razon_social:
                print(f'✓ RUC VÁLIDO ENCONTRADO: {ruc.numero}')
                return EmpresaInfo(ruc=ruc, razon_social=razon_social)
            else:
                print(f'✗ RUC inválido o no encontrado: {ruc.numero}')
        
        print('\n⚠ Ningún RUC fue válido en el web scraping')
        return None


# ==================== RESULT PRINTER ====================

class ResultPrinter:
    """Responsable de imprimir resultados"""
    
    @staticmethod
    def print_header(title: str):
        """Imprime un encabezado formateado"""
        print(f'\n{"="*50}')
        print(f'=== {title} ===')
        print(f'{"="*50}')
    
    @staticmethod
    def print_ocr_texts(ocr_result):
        """Imprime todos los textos detectados por OCR"""
        ResultPrinter.print_header("PALABRAS DETECTADAS")
        
        for res in ocr_result:
            textos = res['rec_texts']
            confianzas = res['rec_scores']
            page = res['page_index']
            
            for j, texto in enumerate(textos, 1):
                print(f'{j}. text: {texto} - confianza: {confianzas[j-1]:.4f} - página: {page}')
        
        print(f'{"="*50}')


# ==================== MAIN PROCESSORS ====================

class ComprobanteNumberProcessor:
    """Orquestador para procesar números en comprobantes"""
    
    def __init__(self, input_path: str):
        self.input_path = input_path
        self.ocr_processor = OCRProcessor(OCRConfig())
        self.number_analyzer = NumberAnalyzer(DecimalExtractor())
        self.printer = ResultPrinter()
    
    def process(self) -> str:
        """Procesa el comprobante y retorna el valor más alto como string o cadena vacía"""
        ocr_result = self.ocr_processor.process(self.input_path)
        self.printer.print_ocr_texts(ocr_result)
        
        numeros_detectados = self.number_analyzer.analyze(ocr_result)
        
        if numeros_detectados:
            numero_mas_alto = max(numeros_detectados, key=lambda x: x.valor_numerico)
            return f'{numero_mas_alto.valor_numerico:.2f}'
        else:
            return ''


class ComprobanteDNIProcessor:
    """Orquestador para procesar DNIs en comprobantes"""
    
    def __init__(self, input_path: str):
        self.input_path = input_path
        self.ocr_processor = OCRProcessor(OCRConfig())
        self.dni_extractor = DNIExtractor()
        self.dni_validator = DNIValidator(WebScraperService())
        self.printer = ResultPrinter()
    
    def process(self) -> tuple[str, str]:
        """Procesa el comprobante y retorna (nombre_completo, dni) o cadenas vacías"""
        print("🔍 Procesando OCR...")
        ocr_result = self.ocr_processor.process(self.input_path)
        self.printer.print_ocr_texts(ocr_result)
        
        dnis_encontrados = self.dni_extractor.extract_all_from_results(ocr_result)
        print(f'\n📋 Total de DNIs candidatos encontrados: {len(dnis_encontrados)}')
        
        if not dnis_encontrados:
            return '', ''
        
        persona_valida = self.dni_validator.validate_dnis(dnis_encontrados)
        
        if persona_valida:
            nombre_completo = persona_valida.nombre_completo if persona_valida.nombre_completo else ''
            dni = persona_valida.dni.numero if persona_valida.dni else ''
            return nombre_completo, dni
        else:
            return '', ''


class ComprobanteRUCProcessor:
    """Orquestador para procesar RUCs en comprobantes"""
    
    def __init__(self, input_path: str):
        self.input_path = input_path
        self.ocr_processor = OCRProcessor(OCRConfig())
        self.ruc_extractor = RUCExtractor()
        self.ruc_validator = RUCValidator(WebScraperServiceRUC())
        self.printer = ResultPrinter()
    
    def process(self) -> tuple[str, str]:
        """Procesa el comprobante y retorna (razón_social, ruc) o cadenas vacías"""
        print("🔍 Procesando OCR...")
        ocr_result = self.ocr_processor.process(self.input_path)
        self.printer.print_ocr_texts(ocr_result)
        
        rucs_encontrados = self.ruc_extractor.extract_all_from_results(ocr_result)
        print(f'\n📋 Total de RUCs candidatos encontrados: {len(rucs_encontrados)}')
        
        if not rucs_encontrados:
            return '', ''
        
        empresa_valida = self.ruc_validator.validate_rucs(rucs_encontrados)
        
        if empresa_valida:
            razon_social = empresa_valida.razon_social if empresa_valida.razon_social else ''
            ruc = empresa_valida.ruc.numero if empresa_valida.ruc else ''
            return razon_social, ruc
        else:
            return '', ''


class ComprobanteTipoProcessor:
    """Orquestador para procesar el tipo de comprobante"""
    
    def __init__(self, input_path: str):
        self.input_path = input_path
        self.ocr_processor = OCRProcessor(OCRConfig())
        self.tipo_extractor = TipoComprobanteExtractor()
        self.printer = ResultPrinter()
    
    def process(self) -> str:
        """Procesa el comprobante y retorna el tipo detectado o cadena vacía"""
        print("🔍 Procesando OCR...")
        ocr_result = self.ocr_processor.process(self.input_path)
        self.printer.print_ocr_texts(ocr_result)
        
        tipo_info = self.tipo_extractor.extract_from_results(ocr_result)
        
        if tipo_info:
            return tipo_info.tipo
        else:
            return ''


class ComprobanteFechaProcessor:
    """Orquestador para procesar la fecha de emisión"""
    
    def __init__(self, input_path: str):
        self.input_path = input_path
        self.ocr_processor = OCRProcessor(OCRConfig())
        self.fecha_extractor = FechaExtractor()
        self.printer = ResultPrinter()
    
    def process(self) -> str:
        """Procesa el comprobante y retorna la fecha normalizada o cadena vacía"""
        print("🔍 Procesando OCR...")
        ocr_result = self.ocr_processor.process(self.input_path)
        self.printer.print_ocr_texts(ocr_result)
        
        fecha_info = self.fecha_extractor.extract_from_results(ocr_result)
        
        if fecha_info:
            return fecha_info.fecha_normalizada
        else:
            return ''


# ==================== ORCHESTRATOR CORREGIDO ====================

import concurrent.futures

class ComprobanteOrchestrator:
    @staticmethod
    def process_comprobante(file_path: str) -> ComprobanteResponse:
        """Procesa OCR una vez y extrae info con lógica mejorada de DNI/RUC cliente"""
        
        # 1️⃣ OCR una sola vez
        ocr_result = OCRProcessor(OCRConfig()).process(file_path)
        
        # 2️⃣ Extraer candidatos
        dnis = DNIExtractor.extract_all_from_results(ocr_result)
        rucs = RUCExtractor.extract_all_from_results(ocr_result)
        
        # 3️⃣ Separar RUCs: primero es empresa, segundo es cliente
        ruc_empresa = rucs[0] if len(rucs) >= 1 else None
        ruc_cliente = rucs[1] if len(rucs) >= 2 else None
        
        # 4️⃣ Validar DNI y RUC empresa en paralelo
        with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
            future_dni = executor.submit(
                DNIValidator(WebScraperService()).validate_dnis, dnis
            )
            future_ruc_empresa = executor.submit(
                RUCValidator(WebScraperServiceRUC()).validate_rucs, [ruc_empresa] if ruc_empresa else []
            )
            
            persona_dni = future_dni.result()
            empresa = future_ruc_empresa.result()
        
        # 5️⃣ Si NO hay DNI válido, buscar RUC cliente (segundo RUC)
        persona_ruc = None
        if not persona_dni and ruc_cliente:
            print('\n⚠️  No se encontró DNI válido, intentando con RUC cliente...')
            persona_ruc = RUCValidator(WebScraperServiceRUC()).validate_rucs([ruc_cliente])
        
        # 6️⃣ Determinar información del cliente (priorizar DNI)
        if persona_dni:
            nombre_cliente = persona_dni.nombre_completo if persona_dni.nombre_completo else ''
            identificacion_cliente = persona_dni.dni.numero if persona_dni.dni else ''
        elif persona_ruc:
            nombre_cliente = persona_ruc.razon_social if persona_ruc.razon_social else ''
            identificacion_cliente = persona_ruc.ruc.numero if persona_ruc.ruc else ''
        else:
            nombre_cliente = ''
            identificacion_cliente = ''
        
        # 7️⃣ Extraer otros datos
        tipo_info = TipoComprobanteExtractor.extract_from_results(ocr_result)
        fecha_info = FechaExtractor.extract_from_results(ocr_result)
        numeros = NumberAnalyzer(DecimalExtractor()).analyze(ocr_result)
        
        # 8️⃣ Retornar respuesta
        return ComprobanteResponse(
            nombre_cliente=nombre_cliente,
            dni_o_ruc_cliente=identificacion_cliente,
            tipo_comprobante=tipo_info.tipo if tipo_info else '',
            fecha_emision=fecha_info.fecha_normalizada if fecha_info else '',
            ruc_empresa=empresa.ruc.numero if empresa and empresa.ruc else '',
            nombre_empresa=empresa.razon_social if empresa else '',
            importe_total=f'{max(numeros, key=lambda x: x.valor_numerico).valor_numerico:.2f}' if numeros else ''
        )


# ==================== EJEMPLO DE USO OPTIMIZADO ====================

if __name__ == '__main__':
    input_path = "comprobantes/comprobante/comprobante con dni y ruc (SAN PABLO).pdf"

    # "comprobantes/images/93-hojas-conv-24_png.rf.00924060192cf9366e17ff4af8d8f2dd.jpg"

    # comprobantes/images/08_jpg.rf.808bc0b8bc656f62f9ed4d4236135e9e.jpg
    # comprobantes/comprobante/comprobante sin nombre de cliente 2 (MIFARMA).jpeg

    # "comprobante/comprobante con dni y ruc (SAN PABLO).pdf"
    # "comprobantes/images/08_jpg.rf.808bc0b8bc656f62f9ed4d4236135e9e.jpg"
    
    # ✅ USAR EL ORCHESTRATOR OPTIMIZADO (ejecuta OCR una sola vez)
    print("\n" + "="*60)
    print("PROCESAMIENTO OPTIMIZADO (OCR una sola vez)")
    print("="*60)
    
    orchestrator = ComprobanteOrchestrator()
    resultado = orchestrator.process_comprobante(input_path)
    
    # Mostrar resumen final
    print("\n" + "="*60)
    print("RESUMEN COMPLETO DEL COMPROBANTE")
    print("="*60)
    print(f'Importe Total: {resultado.importe_total}')
    print(f'Cliente: {resultado.nombre_cliente}')
    print(f'DNI/RUC del cliente: {resultado.dni_o_ruc_cliente}')
    print(f'Empresa: {resultado.nombre_empresa}')
    print(f'RUC de Empresa: {resultado.ruc_empresa}')
    print(f'Tipo: {resultado.tipo_comprobante}')
    print(f'Fecha: {resultado.fecha_emision}')
    print("="*60)

Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/franspaxi/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('latin_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/franspaxi/.paddlex/official_models/latin_PP-OCRv5_mobile_rec`.



PROCESAMIENTO OPTIMIZADO (OCR una sola vez)

🔍 Validando DNIs con web scraping...

🔍 Validando RUCs con web scraping...

Intento 1/1 - Probando RUC: 20601725551

Intento 1/1 - Probando DNI: 78887021
✓ RUC VÁLIDO ENCONTRADO: 20601725551
✓ DNI VÁLIDO ENCONTRADO: 78887021

RESUMEN COMPLETO DEL COMPROBANTE
Importe Total: 150.00
Cliente: FRANS EDWARD PAXI JUCHANI
DNI/RUC del cliente: 78887021
Empresa: CLINICA CERRO COLORADO S.A.C.
RUC de Empresa: 20601725551
Tipo: BOLETA DE VENTA ELECTRÓNICA
Fecha: 2025-07-16


## Comprobante OCR - API

In [3]:
from fastapi import FastAPI, UploadFile, File, HTTPException
import nest_asyncio
nest_asyncio.apply()
from fastapi.responses import JSONResponse
from pydantic import BaseModel
from typing import Optional
import shutil
import os
from pathlib import Path
import tempfile
import uvicorn

# ==================== API MODELS ====================

# class ComprobanteResponse(BaseModel):
#     """Modelo de respuesta para el comprobante procesado"""
#     nombre: str
#     dni: str
#     tipo_comprobante: str
#     fecha_emision: str
#     ruc: str
#     nombre_empresa: str
#     importe_total: str


# ==================== FASTAPI APP ====================

app = FastAPI(
    title="API de Procesamiento de Comprobantes OCR",
    description="API para extraer información de comprobantes usando OCR",
    version="1.0.0"
)

# ==================== API ENDPOINTS ====================

@app.post("/procesar-comprobante/", response_model=ComprobanteResponse)
async def procesar_comprobante(
    file: UploadFile = File(..., description="Archivo de comprobante (PDF, JPG, PNG, etc.)")
):
    """
    Procesa un comprobante y extrae toda la información relevante.
    
    - **file**: Archivo del comprobante (formatos soportados: PDF, JPG, JPEG, PNG, BMP, TIFF, WEBP)
    
    Retorna:
    - **nombre**: Nombre completo del cliente (extraído del DNI)
    - **dni**: Número de DNI del cliente
    - **tipo_comprobante**: Tipo de comprobante (FACTURA, BOLETA, etc.)
    - **fecha_emision**: Fecha de emisión (formato YYYY-MM-DD)
    - **ruc**: RUC de la empresa
    - **nombre_empresa**: Razón social de la empresa
    - **importe_total**: Importe total del comprobante
    """
    
    # Validar formato de archivo
    FORMATOS_VALIDOS = {'.pdf', '.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp'}
    file_extension = Path(file.filename).suffix.lower()
    
    if file_extension not in FORMATOS_VALIDOS:
        raise HTTPException(
            status_code=400,
            detail=f"Formato no soportado: {file_extension}. Formatos válidos: {', '.join(FORMATOS_VALIDOS)}"
        )
    
    # Crear archivo temporal
    temp_file = None
    try:
        # Guardar archivo subido en un directorio temporal
        with tempfile.NamedTemporaryFile(delete=False, suffix=file_extension) as temp_file:
            shutil.copyfileobj(file.file, temp_file)
            temp_file_path = temp_file.name
        
        # Procesar comprobante
        resultado = ComprobanteOrchestrator.process_comprobante(temp_file_path)
        
        return resultado
    
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Error al procesar el comprobante: {str(e)}"
        )
    
    finally:
        # Limpiar archivo temporal
        if temp_file:
            try:
                os.unlink(temp_file_path)
            except:
                pass


@app.get("/")
async def root():
    """Endpoint raíz con información de la API"""
    return {
        "mensaje": "API de Procesamiento de Comprobantes OCR",
        "version": "1.0.0",
        "endpoints": {
            "/procesar-comprobante/": "POST - Procesa un comprobante y extrae información",
            "/docs": "GET - Documentación interactiva (Swagger UI)",
            "/redoc": "GET - Documentación alternativa (ReDoc)"
        }
    }


@app.get("/health")
async def health_check():
    """Endpoint para verificar el estado de la API"""
    return {"status": "ok", "message": "API funcionando correctamente"}


# ==================== EXECUTION ====================

if __name__ == "__main__":
    import asyncio
    from threading import Thread
    
    def run_server():
        uvicorn.run(app, host="0.0.0.0", port=8090)
    
    # Ejecutar el servidor en un hilo separado para Jupyter
    server_thread = Thread(target=run_server, daemon=True)
    server_thread.start()
    
    print("🚀 Servidor FastAPI iniciado en http://0.0.0.0:8090")
    print("📖 Documentación disponible en http://0.0.0.0:8090/docs")

🚀 Servidor FastAPI iniciado en http://0.0.0.0:8090
📖 Documentación disponible en http://0.0.0.0:8090/docs


INFO:     Started server process [12443]
INFO:     Waiting for application startup.


INFO:     Application startup complete.
ERROR:    [Errno 48] error while attempting to bind on address ('0.0.0.0', 8090): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
